### Download and make the dataset ready in Kaggle 


In [1]:

# ## uncomment if The zip file of the dataset isn't downloaded,extraced 
# !pip install gdown
# Copy the link. The file ID is the long string of characters between d/ and /view.
#For example, in the URL https://drive.google.com/file/d/1aBcDeFgHiJkLmNoPqRsTuVwXyZ/view?usp=sharing, 
#the file ID is 1aBcDeFgHiJkLmNoPqRsTuVwXyZ
# !mkdir /kaggle/tmp
# !gdown  1pzXpA5Cz0DJmjRsLxlqRNnJq-kOUvojb -O /kaggle/tmp/Labeled_CICMODBUS2023.zip
# !unzip /kaggle/tmp/Labeled_CICMODBUS2023.zip -d /kaggle/working/

# # ## uncomment if the python modules (modbus.py,utils.py ,...) not cloned  and added to the path 

# !git clone https://github.com/hamid-rd/FLBased-ICS-NIDS.git
# import sys
# # Add the repository folder to the Python path
# repo_path = '/kaggle/working/FLBased-ICS-NIDS'
# repo_input_path = '/kaggle/input/training/FLBased-ICS-NIDS'
# dataset_path = '/kaggle/input/training/'

# for path in {repo_path,repo_input_path,dataset_path}:
#     if path not in sys.path:
#         sys.path.append(path)


In [1]:
# To test if every thing is okay (modbus.py class and correct number of founded csv files )
from modbus import ModbusDataset,ModbusFlowStream

# dataset_directory = "/kaggle/working/ModbusDataset" 
# dataset_directory = "/kaggle/input/training/ModbusDataset" 
dataset_directory = "dataset" 

modbus = ModbusDataset(dataset_directory,"ready")
modbus.summary_print()

# Don't forget to save version in kaggle (to save outputs written on the disk (/kaggle/working/))  

 The CIC Modbus Dataset contains network (pcap) captures and attack logs from a simulated substation network.
                The dataset is categorized into two groups: an attack dataset and a benign dataset
                The attack dataset includes network traffic captures that simulate various types of Modbus protocol attacks in a substation environment.
                The attacks are reconnaissance, query flooding, loading payloads, delay response, modify length parameters, false data injection, stacking Modbus frames, brute force write and baseline replay.
                These attacks are based of some techniques in the MITRE ICS ATT&CK framework.
                On the other hand, the benign dataset consists of normal network traffic captures representing legitimate Modbus communication within the substation network.
                The purpose of this dataset is to facilitate research, analysis, and development of intrusion detection systems, anomaly detection algorithms and

### Unsupervised Autoencoder training  

In [2]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
import numpy as np # For standard deviation calculation
from modbus import ModbusDataset,ModbusFlowStream
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix,recall_score
import torch.optim as optim
from torch.utils.data import DataLoader
import time
from utils import load_scalers
import random
from random import SystemRandom
from sklearn.model_selection import train_test_split
import itertools
import torch.nn.init as init
import flwr as fl
import ray
from collections import Counter
from flwr.common import ndarrays_to_parameters

def compute_threshold(mse_values,k=1):

    """
    K-SIGMA
    Computes the anomaly detection threshold (for marking sample as Intrusion if the IS was greater )
    based on the mean and standard deviation of Mean Squared Error (MSE) values.
    Formula: thr = mean(MSE) + std(MSE)
    Args:
    mse_values (torch.Tensor or list/np.array): A tensor or list of MSE values

                            obtained from the validation set.
    Returns:
    float: The calculated threshold.
    float: The calculated std.

    """
    if not isinstance(mse_values, torch.Tensor):
        mse_values = torch.tensor(mse_values, dtype=torch.float32)
    if mse_values.numel() == 0:
        return 0.0
    mean_mse = torch.mean(mse_values)
    std_mse = torch.std(mse_values)
    print("-----------mse_loss mean : ",f"{mean_mse.item():.4f}","std:",f"{std_mse.item():.4f}")
    threshold = mean_mse + k*std_mse
    return threshold.item(),std_mse.item()

def vae_loss_function(recon_x, x, mu, logvar,beta=0.05):
    """
    VAE loss function.
    """
    #equivalent to BCE loss
    BCE = nn.functional.mse_loss(recon_x, x, reduction='sum')
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    # print(BCE.item(),KLD.item())
    return (BCE + beta*KLD)

def _init_weights( module):
    ## for one layer apply Xavier Initialization
    if isinstance(module, nn.Linear):
        init.xavier_normal_(module.weight)
        if module.bias is not None:
            init.zeros_(module.bias)
    return module


In [3]:
# dataset_directory = "/kaggle/input/training/ModbusDataset" # change this to the folder contain benign and attack subdirs
# dataset_directory = "dataset" 
dataset_directory = "./ModbusDataset/" 

modbus = ModbusDataset(dataset_directory,"ready")
modbus.summary_print()

 The CIC Modbus Dataset contains network (pcap) captures and attack logs from a simulated substation network.
                The dataset is categorized into two groups: an attack dataset and a benign dataset
                The attack dataset includes network traffic captures that simulate various types of Modbus protocol attacks in a substation environment.
                The attacks are reconnaissance, query flooding, loading payloads, delay response, modify length parameters, false data injection, stacking Modbus frames, brute force write and baseline replay.
                These attacks are based of some techniques in the MITRE ICS ATT&CK framework.
                On the other hand, the benign dataset consists of normal network traffic captures representing legitimate Modbus communication within the substation network.
                The purpose of this dataset is to facilitate research, analysis, and development of intrusion detection systems, anomaly detection algorithms and

In [4]:

# AutoEncoder (AE)
class AE(nn.Module):
    """
    Encoder: (76-32-16-4-2)
    Decoder: (2-4-16-32-76)
    """
    def __init__(self,input_dim=76):
        super(AE, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(16, 4),
            nn.ReLU(),
            nn.Linear(4, 2),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(2, 4),
            nn.ReLU(),
            nn.Linear(4, 16),
            nn.ReLU(),
            nn.Linear(16, 32),
            nn.ReLU(),
            nn.Linear(32, input_dim),
            nn.Sigmoid()
        )

    def forward(self, x):
        z = self.encoder(x)
        x_recon = self.decoder(z)
        return x_recon


# Variational AutoEncoder (VAE)
class VAE(nn.Module):
    """
    Encoder: (76-16-4-2 for mu and log_var)
    Decoder: (2-4-16-76)
    return x_recon, mu, logvar
    """
    def __init__(self,input_dim=76):
        super(VAE, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 16),
            nn.ReLU(),
            nn.Linear(16, 4),
            nn.ReLU(),
        )
        self.fc_mu = nn.Linear(4, 2)
        self.fc_logvar = nn.Linear(4, 2)
        self.decoder = nn.Sequential(
            nn.Linear(2, 4),
            nn.ReLU(),
            nn.Linear(4, 16),
            nn.ReLU(),
            nn.Linear(16, input_dim),
            nn.Sigmoid()
                    )

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def forward(self, x):
        h = self.encoder(x)
        mu = self.fc_mu(h)
        logvar = self.fc_logvar(h)
        z = self.reparameterize(mu, logvar)
        x_recon = self.decoder(z)
        return x_recon, mu, logvar

    
class AAE_Encoder(nn.Module):
    def __init__(self,input_dim=76):
        """
        Encoder(Generator):(76-16-4-2)
        """
        super(AAE_Encoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 16),
            nn.LeakyReLU(0.2),
            nn.Linear(16, 4),
            nn.LeakyReLU(0.2),
            nn.Linear(4, 2))
    def forward(self, x):
        return self.encoder(x)
class AAE_Decoder(nn.Module):
    def __init__(self,input_dim=76):
        super(AAE_Decoder, self).__init__()
        self.decoder = nn.Sequential(
            nn.Linear(2, 4),
            nn.LeakyReLU(),
            nn.Linear(4, 16),
            nn.LeakyReLU(),
            nn.Linear(16, input_dim),
            nn.Sigmoid()
        )
    def forward(self, x):
        return self.decoder(x)
class AAE_Discriminator(nn.Module):
    def __init__(self):
        super(AAE_Discriminator, self).__init__()
        # corrected to 2-16-4-1
        self.discriminator = nn.Sequential(
            nn.Linear(2, 16),
            nn.LeakyReLU(),
            nn.Linear(16, 4),
            nn.LeakyReLU(),
            nn.Linear(4, 1), 
            nn.Sigmoid()
        )    
    def forward(self, x):
        return self.discriminator(x)
 
class AdversarialAutoencoder(nn.Module):
    def __init__(self):
        super(AdversarialAutoencoder, self).__init__()
        self.encoder = AAE_Encoder()
        self.decoder = AAE_Decoder()
        self.discriminator = AAE_Discriminator()
    def forward(self, x):
        fake_z = self.encoder(x)
        x_recon = self.decoder(fake_z)
        return fake_z,x_recon


In [5]:
from torch.utils.data import Dataset
class InMemoryDataset(Dataset):
    """A simple dataset that serves data from pre-loaded tensors."""
    def __init__(self, data, labels):
        # This dataset holds references to the data, which should already be in shared memory.
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

### Part a: Centralized learning 

##### You can go from here right to the FL part


In [ ]:
def make_dataloader(load_all_chunks=True):

    ## must be called after that the global train_files,val_files and test_files plus loades scalers are initilized
    ## return train, valid and test dataloaders
    if load_all_chunks==True:
        large_chunk_size = modbus.dataset["metadata"]["founded_files_num"]["total_dataset_num"]

        dataset_configs = {
            "train": {"files": train_files},
            "val": {"files": val_files},
            "test": {"files": test_files},
        }
        datasets = {}
        ae_datasets = {}
        dataloaders = {}

        print("Cow Processing datasets...")

        for name, config in dataset_configs.items():
            print(f"  - Creating '{name}' dataset...")
            
            # 1. Create the primary ModbusFlowStream dataset
            datasets[name] = ModbusFlowStream(
                shuffle=False,
                chunk_size=large_chunk_size,
                batch_size=1,
                csv_files=config["files"],
                scalers=loaded_scalers['network-wide']['min_max_scalers'],
                window_size=window_size
            )
            
            # 2. Call __getitem__(0) once to load the entire dataset chunk into memory
            datasets[name].__getitem__(0)

            # 2. Extract the raw tensors
            all_data = datasets[name].current_chunk_data
            all_labels = datasets[name].current_chunk_labels
            
            # all_data.share_memory_()
            # all_labels.share_memory_()
            
            # 4. Create an instance of our SIMPLE dataset using the SHARED tensors.
            shared_dataset = InMemoryDataset(all_data, all_labels)

            # 5. Create the DataLoader from the simple dataset. This will work correctly with workers.
            if name=="train":
                shuffle_samples =True
            else:
                shuffle_samples =False
            # dataloaders[name] = DataLoader(
            #     shared_dataset,
            #     batch_size=64,
            #     shuffle=shuffle_samples,
            #     num_workers=4
            #     persistent_workers=True,
            #     pin_memory=True
            # )
            dataloaders[name] = DataLoader(
                shared_dataset,
                batch_size=64,
                shuffle=shuffle_samples,
                num_workers=0, # You can now use multiple workers effectively.
                persistent_workers=False,
                pin_memory=False
            )
                    
            
            print(f"  - Finished '{name}' dataset.")
        train_dataloader = dataloaders['train']
        val_dataloader = dataloaders['val']
        test_dataloader = dataloaders['test']

    else :
        train_dataloader=DataLoader(ModbusFlowStream( 
            shuffle=True,chunk_size=1,batch_size=64,csv_files=train_files,scalers=loaded_scalers['network-wide']['min_max_scalers'],window_size=window_size
        ),  batch_size=1,shuffle=False)
        val_dataloader=DataLoader(ModbusFlowStream( 
            shuffle=False,chunk_size=1,batch_size=64,csv_files=val_files,scalers=loaded_scalers['network-wide']['min_max_scalers'],window_size=window_size
        ),batch_size=1,shuffle=False)
        test_dataloader=DataLoader(ModbusFlowStream(shuffle=False,chunk_size=1,batch_size=64,csv_files=test_files,scalers=loaded_scalers['network-wide']['min_max_scalers'],window_size=window_size),
                                    batch_size=1,shuffle=False)
    return train_dataloader,val_dataloader,test_dataloader


In [11]:

def train_eval(model,train_dataloader,val_dataloader,test_dataloader,learning_rates= [5e-6,1e-7,5e-5,1e-5,1e-6],
               weight_decays=[1e-5,1e-4,1e-7],shuffle_files=True,num_epochs=20,eval_epoch=4,criterion_method="mse", k_range=[1,3],train_model=True):
    device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model=model.to(device)
    if criterion_method=="bce":
        criterion = nn.BCELoss(reduction='sum').to(device)
        eval_criterion = nn.BCELoss(reduction='none').to(device)
    else: #mse
        criterion = nn.MSELoss(reduction='sum').to(device)
        eval_criterion = nn.MSELoss(reduction='none').to(device)
    best_f1=0 #to save best version of the model during test
    best_recall=0 #to save best version of the model during test

    for lr, wd in itertools.product(learning_rates, weight_decays):
        if model._get_name()=="AdversarialAutoencoder":
            adversarial_criterion= nn.BCELoss(reduction="sum")
            optimizer_D = optim.Adam(model.discriminator.parameters(), lr=lr, weight_decay=wd)
            optimizer_G =  optim.Adam(list(model.encoder.parameters()) + list(model.decoder.parameters()), lr=lr, weight_decay=wd)
        else:
            AE_optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=wd)
            ### new code
            # AE_optimizer = optim.SGD(model.parameters(), lr=lr, weight_decay=wd)

        print(f"\n==================  lr={lr}, wd={wd} ==================")
        if train_model==True:
            model.apply(_init_weights)
        for epoch in range(num_epochs):
            if train_model==True:
                time_1 = time.time()
                model.train()
                train_loss = 0
                ## for AAE
                Discriminator_loss = 0
                for sequences, labels in train_dataloader:
                    sequences=sequences.squeeze().to(device)
                    if labels.sum()!=0:
                        continue
                    if model._get_name()=="AdversarialAutoencoder":
                        # 1) generator loss

                        target_ones= torch.ones(sequences.size(0), 1,device=device,dtype=torch.float)
                        target_zeros= torch.zeros(sequences.size(0), 1,device=device,dtype=torch.float)
                        random_latent = torch.randn(sequences.size(0), 2, device=device)
                        optimizer_G.zero_grad()
                        fake_z,decoded_seq = model(sequences)
                        G_loss = 0.001*adversarial_criterion(model.discriminator(fake_z),target_ones ) + 0.999*criterion(decoded_seq, sequences)
                        G_loss.backward()
                        optimizer_G.step()
                        # 2) discriminator loss
                        optimizer_D.zero_grad()
                        real_loss = adversarial_criterion(model.discriminator(random_latent), target_ones)
                        fake_loss = adversarial_criterion(model.discriminator(fake_z.detach()),  target_zeros)
                        D_loss = 0.5*(real_loss + fake_loss)
                        D_loss.backward()
                        optimizer_D.step()
                        train_loss+=G_loss.item()
                        Discriminator_loss+=D_loss.item()   
                    else:
                        AE_optimizer.zero_grad()
                        if model._get_name()=="AE":
                            recon = model(sequences)
                            loss = criterion(recon, sequences) / sequences.size(0)
                        elif model._get_name()=="VAE" :
                            recon, mu, logvar = model(sequences)
                            loss = vae_loss_function(recon, sequences, mu, logvar) /sequences.size(0)
                        loss.backward()
                        AE_optimizer.step()
                        train_loss += loss.item()
                print(f"Train : time {(time.time()-time_1):.2f} s",
                f"Epoch {epoch+1}")
                if model._get_name()=="AdversarialAutoencoder":
                    print(f"Generator Loss: {train_loss / len(train_dataloader):.4f}",
                        f"Discriminator Loss: {Discriminator_loss / len(train_dataloader):.4f}")
                else:
                    print(f"Train Loss: {train_loss / len(train_dataloader):.4f}")
            # Evaluate part
            if (epoch + 1) % eval_epoch == 0:
                model.eval() 
                all_val_losses = []
                all_val_labels = []
                print(f"--- Running Evaluation for Epoch {epoch+1} lr ={lr} wd {wd} ---")
                with torch.no_grad():
                    for sequences, labels in val_dataloader:
                        sequences = sequences.squeeze().to(device) 
                        if labels.sum()!=0:
                            continue
                        if criterion_method=="bce":
                            ## may test features be greater than 1 after scaling 
                            sequences=torch.clamp(sequences, min=0.0, max=1.0)      
                        if model._get_name()=="AE":
                            recon = model(sequences)
                        elif model._get_name()=="VAE" :
                            recon, _, _ = model(sequences)
                        elif model._get_name()=="AdversarialAutoencoder":
                            _,recon= model(sequences)
                        val_loss = eval_criterion(recon, sequences)
                        if val_loss.dim() > 1:
                            val_loss = val_loss
                        else:
                            val_loss = val_loss.unsqueeze(dim=0)
                            labels = labels.unsqueeze(dim=0)
                        if val_loss.dim()==3:
                            ##GRU : mean of window
                            val_loss = val_loss.mean(dim=1)
                        val_loss = val_loss.sum(dim=1)
                        all_val_losses.extend(val_loss.cpu().numpy())
                        all_val_labels.extend(labels.flatten().cpu().numpy())     
                threshold_1,std_mse = compute_threshold(all_val_losses,k=0)

                all_val_losses = np.array(all_val_losses).squeeze()  
                all_val_labels = np.array(all_val_labels).squeeze()  
                # If intrusion score > threshold, predict 1 (intrusion), else 0 (benign)
                # For FDR, get True Positives (TP) and False Positives (FP)
                
                predictions = (all_val_losses > threshold_1).astype(int)

                accuracy = accuracy_score(all_val_labels, predictions)
                print(f"Val: Accuracy: {accuracy:.4f}  ")
                model.eval() 
                all_test_losses = []
                all_test_labels = []
                with torch.no_grad():
                    for sequences, labels in test_dataloader:
                        sequences = sequences.squeeze().to(device)
                        labels = labels.squeeze().to(device)
                        if criterion_method=="bce":
                            ## may test features be greater than 1 after scaling 
                            sequences=torch.clamp(sequences, min=0.0, max=1.0)
                        if model._get_name()=="AE":
                            recon = model(sequences)
                        elif model._get_name()=="VAE"  :
                            recon, mu, logvar = model(sequences)
                        elif model._get_name()=="AdversarialAutoencoder":
                            _,recon= model(sequences)

                        intrusion_scores = eval_criterion(recon, sequences)
                        if intrusion_scores.dim() > 1:
                            intrusion_scores = intrusion_scores
                        else:
                            intrusion_scores = intrusion_scores.unsqueeze(dim=0)
                            labels = labels.unsqueeze(dim=0)
                        if intrusion_scores.dim()==3:
                            ##GRU : mean of window
                            intrusion_scores = intrusion_scores.mean(dim=1)
                        intrusion_scores = intrusion_scores.sum(dim=1)
                        all_test_losses.extend(intrusion_scores.cpu().numpy())
                        all_test_labels.extend(labels.cpu().numpy())

                all_test_losses = np.array(all_test_losses)
                all_test_labels = np.array(all_test_labels)
                temp_best_recall =best_recall
                temp_best_f1 =best_f1

                for k in k_range:
                    threshold=threshold_1+k*std_mse
                    print(f" K: {k} K-SIGMA Threshold : ---thr {threshold:.4}")
                    predictions = (all_test_losses > threshold).astype(int)
                    binary_test_labels = (all_test_labels != 0).astype(int)

                    # Find the indices where the prediction was incorrect
                    misclassified_indices = np.where(binary_test_labels != predictions)[0]

                    # Get the original labels for those misclassified instances
                    misclassified_original_labels = all_test_labels[misclassified_indices]

                    # To get a summary count of which labels were misclassified
                    print("Counts of : original binary labels",Counter(binary_test_labels),"predicted binary labels",Counter(predictions))
                    print(f"Counts of  original  labels: {dict(sorted(Counter(all_test_labels).items()))}")
                    print(f"Counts of misclassified original labels: {dict(sorted(Counter(misclassified_original_labels).items()))}")
                    accuracy = accuracy_score(binary_test_labels, predictions)
                    f1 = f1_score(binary_test_labels, predictions, zero_division=0)
                    recall = recall_score(binary_test_labels, predictions,zero_division=0)
                    _, fp, _, tp = confusion_matrix(binary_test_labels, predictions, labels=[0, 1]).ravel()
                    # FDR = FP / (FP + TP) 
                    if (fp + tp) == 0:
                        fdr = 0.0 
                    else:
                        fdr = fp / (fp + tp)
                    print(f"Test : Accuracy: {accuracy:.4f} Recall : {recall:.4f} FDR: {fdr:.4f}  F1-score: {f1:.4f}  ")
                    !mkdir best_models -p
                    if f1>best_f1 :
                        best_f1=f1
                    if recall>best_recall:
                        best_recall=recall
                if (best_recall>temp_best_recall or best_f1 > temp_best_f1):
                    if train_model==True:
                        save_path ="best_models/"+model._get_name()+"_f1_"+f"{best_f1:.2f}" +"_recall_"+f"{best_recall:.2f}" +"_.pth"
                        torch.save(model.state_dict(),save_path)
                        print("model",model._get_name(),"is saved in" ,save_path )


#### Train on network-wdie
#### evaluate on comrpomised-scada IED node ied1b during centralized training.

In [8]:
train_files=[col for col in modbus.dataset["benign_dataset_dir"] if col.find("network-wide")!=-1][:]
test_files=[col for col in modbus.dataset["attack_dataset_dir"]["compromised-scada"] if col.find("ied1b")!=-1]


### missed attack logs files for the day 21 for ied1b which can reduce the accuracy.
test_files.remove(dataset_directory+"attack/compromised-scada/ied1b/ied1b-network-captures/ready/vethc76bd3f-6-labeled.csv")
random.seed(42)
random.shuffle(train_files)
random.shuffle(test_files)
val_files = train_files[-4:]
train_files = train_files[:-4]
print("ied1b comp ied attack ->\n test: ",len(test_files),test_files)
print("----------- Network-wide number of csv files -> \n ----------- train :",len(train_files),train_files,"\n -------- valid:",len(val_files),val_files)

ied1b comp ied attack ->
 test:  7 ['./ModbusDataset/attack/compromised-scada/ied1b/ied1b-network-captures/ready/vethc76bd3f-1-labeled.csv', './ModbusDataset/attack/compromised-scada/ied1b/ied1b-network-captures/ready/vethc76bd3f-5-labeled.csv', './ModbusDataset/attack/compromised-scada/ied1b/ied1b-network-captures/ready/vethc76bd3f-3-labeled.csv', './ModbusDataset/attack/compromised-scada/ied1b/ied1b-network-captures/ready/vethc76bd3f-7-labeled.csv', './ModbusDataset/attack/compromised-scada/ied1b/ied1b-network-captures/ready/vethc76bd3f-2-labeled.csv', './ModbusDataset/attack/compromised-scada/ied1b/ied1b-network-captures/ready/vethc76bd3f-4-labeled.csv', './ModbusDataset/attack/compromised-scada/ied1b/ied1b-network-captures/ready/vethc76bd3f-0-labeled.csv']
----------- Network-wide number of csv files -> 
 ----------- train : 15 ['./ModbusDataset/benign/network-wide-pcap-capture/network-wide/ready/network-wide-normal-16-labeled.csv', './ModbusDataset/benign/network-wide-pcap-capture

In [9]:

load_all_chunks=True
window_size=1
loaded_scalers=load_scalers('fitted_scalers')
torch.manual_seed(42)
np.random.seed(42)

Successfully loaded scalers for 'network-wide'


In [10]:
def make_dataloader(load_all_chunks=True):

    ## must be called if global train_files,val_files and test_files plus loades scalers are initilized
    ## return train, valid and test dataloaders
    if load_all_chunks==True:
        large_chunk_size = modbus.dataset["metadata"]["founded_files_num"]["total_dataset_num"]

        dataset_configs = {
            "train": {"files": train_files},
            "val": {"files": val_files},
            "test": {"files": test_files},
        }
        datasets = {}
        ae_datasets = {}
        dataloaders = {}

        print("Cow Processing datasets...")

        for name, config in dataset_configs.items():
            print(f"  - Creating '{name}' dataset...")
            
            # 1. Create the primary ModbusFlowStream dataset
            datasets[name] = ModbusFlowStream(
                shuffle=False,
                chunk_size=large_chunk_size,
                batch_size=1,
                csv_files=config["files"],
                scalers=loaded_scalers['network-wide']['min_max_scalers'],
                window_size=window_size
            )
            
            # 2. Call __getitem__(0) once to load the entire dataset chunk into memory
            datasets[name].__getitem__(0)

            # 2. Extract the raw tensors
            all_data = datasets[name].current_chunk_data
            all_labels = datasets[name].current_chunk_labels
            
            # all_data.share_memory_()
            # all_labels.share_memory_()
            
            # 4. Create an instance of our SIMPLE dataset using the SHARED tensors.
            shared_dataset = InMemoryDataset(all_data, all_labels)

            # 5. Create the DataLoader from the simple dataset. This will work correctly with workers.
            if name=="train":
                shuffle_samples =True
            else:
                shuffle_samples =False
            # dataloaders[name] = DataLoader(
            #     shared_dataset,
            #     batch_size=64,
            #     shuffle=shuffle_samples,
            #     num_workers=4
            #     persistent_workers=True,
            #     pin_memory=True
            # )
            dataloaders[name] = DataLoader(
                shared_dataset,
                batch_size=64,
                shuffle=shuffle_samples,
                num_workers=0, # You can now use multiple workers effectively.
                persistent_workers=False,
                pin_memory=False
            )
                    
            
            print(f"  - Finished '{name}' dataset.")
        train_dataloader = dataloaders['train']
        val_dataloader = dataloaders['val']
        test_dataloader = dataloaders['test']

    else :
        train_dataloader=DataLoader(ModbusFlowStream( 
            shuffle=True,chunk_size=1,batch_size=64,csv_files=train_files,scalers=loaded_scalers['network-wide']['min_max_scalers'],window_size=window_size
        ),  batch_size=1,shuffle=False)
        val_dataloader=DataLoader(ModbusFlowStream( 
            shuffle=False,chunk_size=1,batch_size=64,csv_files=val_files,scalers=loaded_scalers['network-wide']['min_max_scalers'],window_size=window_size
        ),batch_size=1,shuffle=False)
        test_dataloader=DataLoader(ModbusFlowStream(shuffle=False,chunk_size=1,batch_size=64,csv_files=test_files,scalers=loaded_scalers['network-wide']['min_max_scalers'],window_size=window_size),
                                    batch_size=1,shuffle=False)
    return train_dataloader,val_dataloader,test_dataloader


In [47]:
train_dataloader,val_dataloader,test_dataloader=make_dataloader(load_all_chunks==True)


Cow Processing datasets...
  - Creating 'train' dataset...


KeyboardInterrupt: 

In [ ]:
t_1=time.time()
for seq,label in train_dataloader:
     pass
print(time.time()-t_1)

20.64165949821472


In [ ]:
print(len(train_dataloader),len(val_dataloader),len(test_dataloader))

34527 9584 11720


In [ ]:
AE_model = AE(input_dim=76)
train_eval(AE_model,train_dataloader,val_dataloader,test_dataloader,shuffle_files=True,num_epochs=6,eval_epoch=2,criterion_method="mse",learning_rates=[5e-3,1e-3,5e-4,1e-4],weight_decays=[1e-5,1e-6],k_range=[1,3])
# --- Running Evaluation for Epoch 6 lr =0.001 wd 1e-05 ---



==================  lr=0.005, wd=1e-05 ==================
Train : time 156.91 s Epoch 1
Train Loss: 0.1165
Train : time 154.89 s Epoch 2
Train Loss: 0.0165
--- Running Evaluation for Epoch 2 lr =0.005 wd 1e-05 ---
-----------mse_loss mean :  0.0154 std: 0.2341
Val: Accuracy: 0.9908  
 K: 1 K-SIGMA Threshold : ---thr 0.2495
Counts of : original binary labels Counter({0: 529457, 1: 220322}) predicted binary labels Counter({0: 561817, 1: 187962})
Counts of  original  labels: {0: 529457, 1: 218884, 2: 317, 3: 236, 4: 243, 5: 240, 6: 170, 7: 232}
Counts of misclassified original labels: {0: 7941, 1: 40034, 2: 58, 3: 96, 4: 42, 5: 2, 6: 68, 7: 1}
Test : Accuracy: 0.9357 Recall : 0.8171 FDR: 0.0422  F1-score: 0.8818  
 K: 3 K-SIGMA Threshold : ---thr 0.7176
Counts of : original binary labels Counter({0: 529457, 1: 220322}) predicted binary labels Counter({0: 731959, 1: 17820})
Counts of  original  labels: {0: 529457, 1: 218884, 2: 317, 3: 236, 4: 243, 5: 240, 6: 170, 7: 232}
Counts of miscla

In [ ]:
VAE_model = VAE(input_dim=76)
train_eval(VAE_model,train_dataloader,val_dataloader,test_dataloader,shuffle_files=True,num_epochs=6,eval_epoch=2,criterion_method="mse",learning_rates=[1e-3,1e-4],weight_decays=[1e-5,1e-6],k_range=[1,3])



==================  lr=0.001, wd=1e-05 ==================
Train : time 160.35 s Epoch 1
Train Loss: 0.2666
Train : time 154.92 s Epoch 2
Train Loss: 0.1515
--- Running Evaluation for Epoch 2 lr =0.001 wd 1e-05 ---
-----------mse_loss mean :  0.0407 std: 0.2427
Val: Accuracy: 0.8383  
 K: 1 K-SIGMA Threshold : ---thr 0.2834
Counts of : original binary labels Counter({0: 529457, 1: 220322}) predicted binary labels Counter({0: 557414, 1: 192365})
Counts of  original  labels: {0: 529457, 1: 218884, 2: 317, 3: 236, 4: 243, 5: 240, 6: 170, 7: 232}
Counts of misclassified original labels: {0: 11974, 1: 39666, 2: 58, 3: 94, 4: 41, 5: 2, 6: 67, 7: 3}
Test : Accuracy: 0.9308 Recall : 0.8188 FDR: 0.0622  F1-score: 0.8742  
 K: 3 K-SIGMA Threshold : ---thr 0.7688
Counts of : original binary labels Counter({0: 529457, 1: 220322}) predicted binary labels Counter({0: 562932, 1: 186847})
Counts of  original  labels: {0: 529457, 1: 218884, 2: 317, 3: 236, 4: 243, 5: 240, 6: 170, 7: 232}
Counts of misc

KeyboardInterrupt: 

In [ ]:
AAE_model = AdversarialAutoencoder()
train_eval(AAE_model,train_dataloader,val_dataloader,test_dataloader,shuffle_files=True,num_epochs=6,eval_epoch=3,criterion_method="mse",learning_rates=[1e-3,1e-4],weight_decays=[1e-5,1e-6])



==================  lr=0.001, wd=1e-05 ==================
Train : time 231.25 s Epoch 1
Generator Loss: 7.7712 Discriminator Loss: 13.8002
Train : time 239.30 s Epoch 2
Generator Loss: 1.1697 Discriminator Loss: 18.9262
Train : time 253.08 s Epoch 3
Generator Loss: 1.1140 Discriminator Loss: 22.2141
--- Running Evaluation for Epoch 3 lr =0.001 wd 1e-05 ---
-----------mse_loss mean :  0.0155 std: 0.2409
Val: Accuracy: 0.9942  
 K: 1 K-SIGMA Threshold : ---thr 0.2564
Counts of : original binary labels Counter({0: 529457, 1: 220322}) predicted binary labels Counter({0: 568636, 1: 181143})
Counts of  original  labels: {0: 529457, 1: 218884, 2: 317, 3: 236, 4: 243, 5: 240, 6: 170, 7: 232}
Counts of misclassified original labels: {0: 7874, 1: 46780, 2: 61, 3: 99, 4: 40, 5: 2, 6: 70, 7: 1}
Test : Accuracy: 0.9267 Recall : 0.7864 FDR: 0.0435  F1-score: 0.8632  
 K: 3 K-SIGMA Threshold : ---thr 0.7381
Counts of : original binary labels Counter({0: 529457, 1: 220322}) predicted binary labels Co

#### Evaluate pre-trained autoencoders  on the compromised-ied and compromised scada scenarios 

No exact labeling for the comp ied scenario results in low performance 

In [ ]:
Trained_AE_model=AE(input_dim=76)
Trained_AE_model.load_state_dict(torch.load("./best_models/AE_f1_0.88_recall_0.82_.pth"))
Trained_VAE_model=VAE(input_dim=76)
Trained_VAE_model.load_state_dict(torch.load("./best_models/VAE_f1_0.88_recall_0.82_.pth"))
Trained_AAE_model=AdversarialAutoencoder()
Trained_AAE_model.load_state_dict(torch.load("./best_models/AdversarialAutoencoder_f1_0.88_recall_0.82_.pth"))

<All keys matched successfully>

In [ ]:
dataset_directory = "./ModbusDataset" 

modbus = ModbusDataset(dataset_directory,"ready")
modbus.summary_print()


 The CIC Modbus Dataset contains network (pcap) captures and attack logs from a simulated substation network.
                The dataset is categorized into two groups: an attack dataset and a benign dataset
                The attack dataset includes network traffic captures that simulate various types of Modbus protocol attacks in a substation environment.
                The attacks are reconnaissance, query flooding, loading payloads, delay response, modify length parameters, false data injection, stacking Modbus frames, brute force write and baseline replay.
                These attacks are based of some techniques in the MITRE ICS ATT&CK framework.
                On the other hand, the benign dataset consists of normal network traffic captures representing legitimate Modbus communication within the substation network.
                The purpose of this dataset is to facilitate research, analysis, and development of intrusion detection systems, anomaly detection algorithms and

In [ ]:
for scenario in {"external"}:
    if scenario=="compromised-scada":
        print("scenario :",scenario)
        test_files= [col for col in modbus.dataset["attack_dataset_dir"][scenario] if col.find("ied4c")!=-1]
    elif scenario=="compromised-ied":
        print("scenario :",scenario)
        test_files= [col for col in modbus.dataset["attack_dataset_dir"][scenario] if col.find("ied1b")!=-1]
    else:
        print("scenario :",scenario)
        test_files= [col for col in modbus.dataset["attack_dataset_dir"][scenario] if col.find("substation-wide-capture")!=-1]        

    print("----------- benign valid files:",len(val_files),val_files)
    print(f"----------{scenario} attack  test files : ",len(test_files),test_files)
    val_dataloader=DataLoader(ModbusFlowStream(
                shuffle=False,
                chunk_size=100,
                batch_size=64,
                csv_files=val_files,
                scalers=loaded_scalers['network-wide']['min_max_scalers'],
            ),batch_size=1,shuffle=False)
    test_dataloader=DataLoader(ModbusFlowStream(
                shuffle=False,
                chunk_size=100,
                batch_size=64,
                csv_files=test_files,
                scalers=loaded_scalers['network-wide']['min_max_scalers'],
            ),batch_size=1,shuffle=False)
    for trained_model in {Trained_AE_model,Trained_VAE_model,Trained_AAE_model}:
        print("*"*10,trained_model._get_name(),10*"*")
        train_eval(trained_model,None,val_dataloader,test_dataloader,shuffle_files=False,num_epochs=1,eval_epoch=1,criterion_method="mse",train_model=False,learning_rates=[0],weight_decays=[0])
        

scenario : compromised-scada
----------- benign valid files: 4 ['./ModbusDataset/benign/network-wide-pcap-capture/network-wide/ready/network-wide-normal-28-labeled.csv', './ModbusDataset/benign/network-wide-pcap-capture/network-wide/ready/network-wide-normal-22-labeled.csv', './ModbusDataset/benign/network-wide-pcap-capture/network-wide/ready/network-wide-normal-25-labeled.csv', './ModbusDataset/benign/network-wide-pcap-capture/network-wide/ready/network-wide-normal-17-labeled.csv']
----------compromised-scada attack  test files :  6 ['./ModbusDataset/attack/compromised-scada/ied4c/ied4c-network-captures/ready/vethe685ac9-0-labeled.csv', './ModbusDataset/attack/compromised-scada/ied4c/ied4c-network-captures/ready/vethe685ac9-5-labeled.csv', './ModbusDataset/attack/compromised-scada/ied4c/ied4c-network-captures/ready/vethe685ac9-1-labeled.csv', './ModbusDataset/attack/compromised-scada/ied4c/ied4c-network-captures/ready/vethe685ac9-2-labeled.csv', './ModbusDataset/attack/compromised-sca

/home/hamid_rd3/labeling/Project1404/FLBased-ICS-NIDS/vnv/lib/python3.10/site-packages/numpy/lib/function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/home/hamid_rd3/labeling/Project1404/FLBased-ICS-NIDS/vnv/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


 K: 3 K-SIGMA Threshold : ---thr 0.7108
Counts of : original binary labels Counter() predicted binary labels Counter()
Counts of  original  labels: {}
Counts of misclassified original labels: {}
Test : Accuracy: nan Recall : 0.0000 FDR: 0.0000  F1-score: 0.0000  
********** AE **********

==================  lr=0, wd=0 ==================
--- Running Evaluation for Epoch 1 lr =0 wd 0 ---


/home/hamid_rd3/labeling/Project1404/FLBased-ICS-NIDS/vnv/lib/python3.10/site-packages/numpy/lib/function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/home/hamid_rd3/labeling/Project1404/FLBased-ICS-NIDS/vnv/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


-----------mse_loss mean :  0.0049 std: 0.1302
Val: Accuracy: 0.9907  
 K: 1 K-SIGMA Threshold : ---thr 0.1351
Counts of : original binary labels Counter() predicted binary labels Counter()
Counts of  original  labels: {}
Counts of misclassified original labels: {}
Test : Accuracy: nan Recall : 0.0000 FDR: 0.0000  F1-score: 0.0000  


/home/hamid_rd3/labeling/Project1404/FLBased-ICS-NIDS/vnv/lib/python3.10/site-packages/numpy/lib/function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/home/hamid_rd3/labeling/Project1404/FLBased-ICS-NIDS/vnv/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/home/hamid_rd3/labeling/Project1404/FLBased-ICS-NIDS/vnv/lib/python3.10/site-packages/numpy/lib/function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/home/hamid_rd3/labeling/Project1404/FLBased-ICS-NIDS/vnv/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


 K: 3 K-SIGMA Threshold : ---thr 0.3956
Counts of : original binary labels Counter() predicted binary labels Counter()
Counts of  original  labels: {}
Counts of misclassified original labels: {}
Test : Accuracy: nan Recall : 0.0000 FDR: 0.0000  F1-score: 0.0000  
********** AdversarialAutoencoder **********

==================  lr=0, wd=0 ==================
--- Running Evaluation for Epoch 1 lr =0 wd 0 ---
-----------mse_loss mean :  0.0056 std: 0.0996
Val: Accuracy: 0.9444  
 K: 1 K-SIGMA Threshold : ---thr 0.1051
Counts of : original binary labels Counter() predicted binary labels Counter()
Counts of  original  labels: {}
Counts of misclassified original labels: {}
Test : Accuracy: nan Recall : 0.0000 FDR: 0.0000  F1-score: 0.0000  


/home/hamid_rd3/labeling/Project1404/FLBased-ICS-NIDS/vnv/lib/python3.10/site-packages/numpy/lib/function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/home/hamid_rd3/labeling/Project1404/FLBased-ICS-NIDS/vnv/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


 K: 3 K-SIGMA Threshold : ---thr 0.3042
Counts of : original binary labels Counter() predicted binary labels Counter()
Counts of  original  labels: {}
Counts of misclassified original labels: {}
Test : Accuracy: nan Recall : 0.0000 FDR: 0.0000  F1-score: 0.0000  


/home/hamid_rd3/labeling/Project1404/FLBased-ICS-NIDS/vnv/lib/python3.10/site-packages/numpy/lib/function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/home/hamid_rd3/labeling/Project1404/FLBased-ICS-NIDS/vnv/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [ ]:
for scenario in {"external","compromised-scada"}:
    if scenario=="compromised-scada":
        print("scenario :",scenario)
        test_files= [col for col in modbus.dataset["attack_dataset_dir"][scenario] if col.find("ied1a")!=-1]
    elif scenario=="compromised-ied":
        print("scenario :",scenario)
        test_files= [col for col in modbus.dataset["attack_dataset_dir"][scenario] if col.find("ied1a")!=-1]
    else:
        print("scenario :",scenario)
        test_files= [col for col in modbus.dataset["attack_dataset_dir"][scenario] if col.find("network-wide")!=-1]        

    print("----------- benign valid files:",len(val_files),val_files)
    print(f"----------{scenario} attack  test files : ",len(test_files),test_files)
    val_dataloader=DataLoader(ModbusFlowStream(
                shuffle=False,
                chunk_size=100,
                batch_size=64,
                csv_files=val_files,
                scalers=loaded_scalers['network-wide']['min_max_scalers'],
            ),batch_size=1,shuffle=False)
    test_dataloader=DataLoader(ModbusFlowStream(
                shuffle=False,
                chunk_size=100,
                batch_size=64,
                csv_files=test_files,
                scalers=loaded_scalers['network-wide']['min_max_scalers'],
            ),batch_size=1,shuffle=False)
    for trained_model in {Trained_AE_model,Trained_VAE_model,Trained_AAE_model}:
        print("*"*10,trained_model._get_name(),10*"*")
        train_eval(trained_model,None,val_dataloader,test_dataloader,shuffle_files=False,num_epochs=1,eval_epoch=1,criterion_method="mse",train_model=False,learning_rates=[0],weight_decays=[0])
        

scenario : compromised-scada
----------- benign valid files: 4 ['./ModbusDataset/benign/network-wide-pcap-capture/network-wide/ready/network-wide-normal-28-labeled.csv', './ModbusDataset/benign/network-wide-pcap-capture/network-wide/ready/network-wide-normal-22-labeled.csv', './ModbusDataset/benign/network-wide-pcap-capture/network-wide/ready/network-wide-normal-25-labeled.csv', './ModbusDataset/benign/network-wide-pcap-capture/network-wide/ready/network-wide-normal-17-labeled.csv']
----------compromised-scada attack  test files :  6 ['./ModbusDataset/attack/compromised-scada/ied1a/ied1a-network-captures/ready/veth3614724-3-labeled.csv', './ModbusDataset/attack/compromised-scada/ied1a/ied1a-network-captures/ready/veth3614724-4-labeled.csv', './ModbusDataset/attack/compromised-scada/ied1a/ied1a-network-captures/ready/veth3614724-0-labeled.csv', './ModbusDataset/attack/compromised-scada/ied1a/ied1a-network-captures/ready/veth3614724-1-labeled.csv', './ModbusDataset/attack/compromised-sca

### Part b: Federated learning 
####  non iid distribution of dataset (ip\node based)

In [ ]:
# ==============================================================================
# 1. SETUP: INSTALL LIBRARIES AND IMPORT DEPENDENCIES
# ==============================================================================
# In a Kaggle notebook, run this cell first to install the necessary libraries.
# !pip install -q flwr[simulation] torch torchvision pandas scikit-learn matplotlib seaborn


In [13]:

import flwr as fl
from collections import OrderedDict
from typing import Dict, List, Tuple, Optional
import seaborn as sns
import os 
from flwr.common import Context # Make sure this import is added
import random
# Suppress warning messages for a cleaner output
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"
# Set a seed for reproducibility
SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

#global device
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Training on {DEVICE}")


Training on cuda:0


In [14]:

# ==============================================================================
#  FEDERATED LEARNING CLIENT: FlowerClient
# ==============================================================================
class FlowerClient(fl.client.NumPyClient):
    """Flower client for training."""
    def __init__(self, cid, model, trainloader):
        self.cid = cid
        self.model = model
        self.train_dataloader = trainloader

    def get_parameters(self, config):
        return [val.cpu().numpy() for _, val in self.model.state_dict().items()]

    def set_parameters(self, parameters):
        params_dict = zip(self.model.state_dict().keys(), parameters)
        state_dict = OrderedDict({k: torch.tensor(v) for k, v in params_dict})
        self.model.load_state_dict(state_dict, strict=True)

    def fit(self, parameters, config):
        self.set_parameters(parameters)
        model =self.model
        lr = cfg.LEARNING_RATE
        wd= cfg.WEIGHT_DECAY
        
        criterion = nn.MSELoss(reduction='sum').to(DEVICE)
        if model._get_name()=="AdversarialAutoencoder":
            adversarial_criterion= nn.BCELoss(reduction="sum")
            optimizer_D = optim.Adam(model.discriminator.parameters(), lr=lr, weight_decay=wd)
            optimizer_G =  optim.Adam(list(model.encoder.parameters()) + list(model.decoder.parameters()), lr=lr, weight_decay=wd)
        else:
            AE_optimizer = optim.Adam(model.parameters(), lr=lr,weight_decay=wd)

        if cfg.STRATEGY == "FED_PROX":
            global_params_dict = {
                k: torch.tensor(v, device=DEVICE) 
                for k, v in zip(self.model.state_dict().keys(), parameters)
            }

        for epoch in range(cfg.LOCAL_EPOCHS):
                time_1 = time.time()
                model.train()
                train_loss = 0
                ## for AAE
                Discriminator_loss = 0
                for sequences, _ in self.train_dataloader:
                    sequences=sequences.squeeze().to(DEVICE)
                    if model._get_name()=="AdversarialAutoencoder":
                        target_ones= torch.ones(sequences.size(0), 1,device=DEVICE,dtype=torch.float)
                        target_zeros= torch.zeros(sequences.size(0), 1,device=DEVICE,dtype=torch.float)
                        random_latent = torch.randn(sequences.size(0), 2, device=DEVICE)
                        optimizer_G.zero_grad()
                        fake_z,decoded_seq = model(sequences)
                        G_loss = 0.001*adversarial_criterion(model.discriminator(fake_z),target_ones ) + 0.999*criterion(decoded_seq, sequences)
                        if cfg.STRATEGY == "FED_PROX":
                            proximal_term_G = 0.0
                            # Proximal term for ENCODER
                            for name, local_param in model.encoder.named_parameters():
                                global_param = global_params_dict['encoder.' + name]
                                proximal_term_G += torch.pow((local_param - global_param).norm(2), 2)
                            # Proximal term for DECODER
                            for name, local_param in model.decoder.named_parameters():
                                global_param = global_params_dict['decoder.' + name]
                                proximal_term_G += torch.pow((local_param - global_param).norm(2), 2)
                            
                            G_loss += (cfg.PROXIMAL_MU / 2) * proximal_term_G
                    
                        G_loss.backward()
                        optimizer_G.step()
                        # 2) discriminator loss
                        optimizer_D.zero_grad()
                        real_loss = adversarial_criterion(model.discriminator(random_latent), target_ones)
                        fake_loss = adversarial_criterion(model.discriminator(fake_z.detach()),  target_zeros)
                        D_loss =  0.5*(real_loss + fake_loss)
                        if cfg.STRATEGY == "FED_PROX":
                            proximal_term_D = 0.0
                            # Proximal term for DISCRIMINATOR
                            for name, local_param in model.discriminator.named_parameters():
                                global_param = global_params_dict['discriminator.' + name]
                                proximal_term_D += torch.pow((local_param - global_param).norm(2), 2)
                            D_loss += (cfg.PROXIMAL_MU / 2) * proximal_term_D
                
                        D_loss.backward()
                        optimizer_D.step()
                        train_loss+=G_loss.item()
                        Discriminator_loss+=D_loss.item()   
                    else:
                        AE_optimizer.zero_grad()
                        if model._get_name()=="AE":
                            recon = model(sequences)
                            loss = criterion(recon, sequences) / sequences.size(0)
                        elif model._get_name()=="VAE" :
                            recon, mu, logvar = model(sequences)
                            loss = vae_loss_function(recon, sequences, mu, logvar) /sequences.size(0)
                        if cfg.STRATEGY == "FED_PROX":
                            proximal_term = 0.0
                            for name, local_param in model.encoder.named_parameters():
                                global_param = global_params_dict['encoder.' + name]
                                proximal_term += torch.pow((local_param - global_param).norm(2), 2)
                            # Proximal term for DECODER
                            for name, local_param in model.decoder.named_parameters():
                                global_param = global_params_dict['decoder.' + name]
                                proximal_term += torch.pow((local_param - global_param).norm(2), 2)
                            loss+=proximal_term
                        loss.backward()
                        AE_optimizer.step()
                        train_loss += loss.item()
                print(f"Train : time {(time.time()-time_1):.2f} s",
                f"Epoch {epoch+1}")
                if model._get_name()=="AdversarialAutoencoder":
                    print(f"Generator Loss: {train_loss / len(self.train_dataloader):.4f}",
                        f"Discriminator Loss: {Discriminator_loss / len(self.train_dataloader):.4f}")
                else:
                    print(f"Train Loss: {train_loss / len(self.train_dataloader):.4f}")
        return self.get_parameters(config={}), len(self.train_dataloader.dataset), {}

    def evaluate(self, parameters, config):
        return 0.0, 0, {}


In [15]:

# ==============================================================================
#  SERVER-SIDE LOGIC AND SIMULATION START
# ==============================================================================

def client_function(context:Context ) -> FlowerClient:
    client_id = int(context.node_config["partition-id"])
    trainloader = load_data_from_id(client_id,"client",chunk_size=1)
    model = get_model().to(DEVICE)
    return FlowerClient(client_id, model, trainloader).to_client()

def make_client_fn_with_cache(chunk_size=10000):
    print("...")
    dataloader_cache: Dict[str, DataLoader] = {}
    def client_fn(context:Context ) -> FlowerClient:
        client_id = int(context.node_config["partition-id"])
        if client_id not in dataloader_cache:
            # If not, create it once and store it in the cache
            print(f"Round 1: Loading and caching data for client {client_id}...")
            dataloader_cache[client_id] = load_data_from_id(client_id,"client",chunk_size=chunk_size)
        else:
            print(f"Reusing cached dataloader for client {client_id}...")
        trainloader = dataloader_cache[client_id]
        model = get_model().to(DEVICE)
        return FlowerClient(client_id, model, trainloader).to_client()
    return client_fn


def get_evaluate_fn():
    """Return an evaluation function for server-side evaluation with caching """
    val_dataloader = load_data_from_id(0,"server",chunk_size=1)
    test_dataloader = load_data_from_id(1,"server",chunk_size=1)
    eval_criterion = nn.MSELoss(reduction='none').to(DEVICE)
    best_f1=0
    best_recall=0

    def evaluate(
        server_round: int,
        parameters: fl.common.NDArrays,
        config: Dict[str, fl.common.Scalar],
        train_model=True
    ) -> Optional[Tuple[float, Dict[str, fl.common.Scalar]]]:
        nonlocal best_f1,best_recall
        model = get_model() # Use the get_model function
        params_dict = zip(model.state_dict().keys(), parameters)
        state_dict = OrderedDict({k: torch.tensor(v) for k, v in params_dict})
        model.load_state_dict(state_dict, strict=True)
        model.to(DEVICE)
        model.eval()
        # Evaluate part
        all_val_losses = []
        all_val_labels = []
        print(f"--- Running Evaluation for Server round {server_round} ---")
        with torch.no_grad():
            for sequences, labels in val_dataloader:
                sequences = sequences.squeeze().to(DEVICE) 
                if labels.sum()!=0:
                    continue
                if model._get_name()=="AE":
                    recon = model(sequences)
                elif model._get_name()=="VAE" :
                    recon, _, _ = model(sequences)
                elif model._get_name()=="AdversarialAutoencoder":
                    _,recon= model(sequences)
                val_loss = eval_criterion(recon, sequences)
                if val_loss.dim() > 1:
                    val_loss = val_loss
                else:
                    val_loss = val_loss.unsqueeze(dim=0)
                    labels = labels.unsqueeze(dim=0)
                if val_loss.dim()==3:
                    ##GRU : mean of window
                    val_loss = val_loss.mean(dim=1)
                val_loss = val_loss.sum(dim=1)
                all_val_losses.extend(val_loss.cpu().numpy())
                all_val_labels.extend(labels.flatten().cpu().numpy())     
        threshold_1,std_mse = compute_threshold(all_val_losses,k=0)

        all_val_losses = np.array(all_val_losses).squeeze()  
        all_val_labels = np.array(all_val_labels).squeeze()  
        # If intrusion score > threshold, predict 1 (intrusion), else 0 (benign)
        # For FDR, get True Positives (TP) and False Positives (FP)
        
        predictions = (all_val_losses > threshold_1).astype(int)

        accuracy = accuracy_score(all_val_labels, predictions)
        print(f"Val: Accuracy: {accuracy:.4f}  ")
        model.eval() 

        all_test_losses = []
        all_test_labels = []
        temp_best_recall =best_recall
        temp_best_f1 =best_f1
        with torch.no_grad():
            for sequences, labels in test_dataloader:
                sequences = sequences.squeeze().to(DEVICE)
                labels = labels.squeeze().to(DEVICE)
                if model._get_name()=="AE":
                    recon = model(sequences)
                elif model._get_name()=="VAE" :
                    recon, mu, logvar = model(sequences)
                elif model._get_name()=="AdversarialAutoencoder":
                    _,recon= model(sequences)

                intrusion_scores = eval_criterion(recon, sequences)
                if intrusion_scores.dim() > 1:
                    intrusion_scores = intrusion_scores
                else:
                    intrusion_scores = intrusion_scores.unsqueeze(dim=0)
                    labels = labels.unsqueeze(dim=0)
                if intrusion_scores.dim()==3:
                    ##GRU : mean of window
                    intrusion_scores = intrusion_scores.mean(dim=1)
                intrusion_scores = intrusion_scores.sum(dim=1)
                all_test_losses.extend(intrusion_scores.cpu().numpy())
                all_test_labels.extend(labels.cpu().numpy())

        all_test_losses = np.array(all_test_losses)
        all_test_labels = np.array(all_test_labels)

        test_result = {}
        for k in {1,3}:
            threshold=threshold_1+k*std_mse
            print(f" K: {k} K-SIGMA Threshold : ---thr {threshold:.4}")
            predictions = (all_test_losses > threshold).astype(int)
            binary_test_labels = (all_test_labels != 0).astype(int)

            # Find the indices where the prediction was incorrect
            misclassified_indices = np.where(binary_test_labels != predictions)[0]

            # Get the original labels for those misclassified instances
            misclassified_original_labels = all_test_labels[misclassified_indices]

            # To get a summary count of which labels were misclassified
            print("Counts of : original binary labels",Counter(binary_test_labels),"predicted binary labels",Counter(predictions))
            print(f"Counts of  original  labels: {dict(sorted(Counter(all_test_labels).items()))}")
            print(f"Counts of misclassified original labels: {dict(sorted(Counter(misclassified_original_labels).items()))}")
            accuracy = accuracy_score(binary_test_labels, predictions)
            f1 = f1_score(binary_test_labels, predictions, zero_division=0)
            recall = recall_score(binary_test_labels, predictions,zero_division=0)
            _, fp, _, tp = confusion_matrix(binary_test_labels, predictions, labels=[0, 1]).ravel()
            # FDR = FP / (FP + TP) 
            if (fp + tp) == 0:
                fdr = 0.0 
            else:
                fdr = fp / (fp + tp)
            test_result[k] = f"k= {k} ,Test : Accuracy: {accuracy:.4f} Recall : {recall:.4f} FDR: {fdr:.4f}  F1-score: {f1:.4f} "
            print(test_result[k])
            !mkdir fed_best_models -p
            if f1>best_f1 :
                best_f1=f1
            if recall>best_recall:
                best_recall=recall
        if (best_recall>temp_best_recall or best_f1 > temp_best_f1):
            if train_model:
                save_path ="fed_best_models/"+cfg.STRATEGY+"_"+model._get_name()+"_f1_"+f"{best_f1:.2f}" +"_recall_"+f"{best_recall:.2f}" +"_.pth"
                torch.save(model.state_dict(),save_path)
                print("model",model._get_name(),"is saved in" ,save_path )
        return np.sum(all_test_losses)/len(all_test_losses),test_result

    return evaluate


In [16]:

def get_initial_parameters(model_name: str):
    """
    Initializes the model weights using Xavier uniform distribution
    and returns them as a Flower Parameters object.
    """
    
    temp_model = get_model()
    for param in temp_model.parameters():
        if param.dim() > 1:
            nn.init.xavier_uniform_(param)
            
    ndarrays = [val.cpu().numpy() for _, val in temp_model.state_dict().items()]
    return ndarrays_to_parameters(ndarrays)


def load_data_from_id(id: int, node = "client" ,chunk_size=10000):
    """Loads the data for a specific training client."""
    if node == "client":
        file_list = TRAIN_CLIENT_DATA_MAPPING[id]
        shuffle=cfg.SHUFFLE_FILES
    else: # server
        file_list = SERVER_EVALUATION_DATA_MAPPING[id]
        shuffle = False
    
    ## means load all chunks of data in memory at once 
    if chunk_size==10000:
        train_loader=DataLoader(ModbusFlowStream(
                shuffle=False,
                chunk_size=chunk_size,
                batch_size=1 ,
                csv_files=file_list,
                scalers=loaded_scalers['network-wide']['min_max_scalers'],
            ),batch_size=64,shuffle=shuffle)
    else :
        train_loader=DataLoader(ModbusFlowStream(
                shuffle=shuffle,
                chunk_size=chunk_size,
                batch_size=cfg.BATCH_SIZE ,
                csv_files=file_list,
                scalers=loaded_scalers['network-wide']['min_max_scalers'],
            ),batch_size=1,shuffle=False)
    return train_loader

def get_model():
    """Returns the model specified in the config."""
    if cfg.MODEL_NAME == "VAE":
        print(f"Using Variational Autoencoder (VAE) ")
        return VAE(input_dim=cfg.INPUT_DIM)
    elif cfg.MODEL_NAME == "AE":
        print(f"Using Autoencoder (AE) ")
        return AE(input_dim=cfg.INPUT_DIM)
    elif cfg.MODEL_NAME =="AAE":
        print(f"Using Adverserial Autoencoder (AAE) ")
        return AdversarialAutoencoder()#76
    else:
        raise ValueError(f"Unknown model name: {cfg.MODEL_NAME}. Choose 'AE' or 'VAE' or 'AAE'.")

def set_server_strategy():
    evaluate_function = get_evaluate_fn()

    if cfg.STRATEGY == "FED_PROX":
        strategy = fl.server.strategy.FedProx(
            fraction_fit=1.0, fraction_evaluate=0.0,
            min_fit_clients=cfg.NUM_TRAIN_CLIENTS,
            min_available_clients=cfg.NUM_TRAIN_CLIENTS,
            evaluate_fn=evaluate_function,
            proximal_mu=cfg.PROXIMAL_MU,
            initial_parameters=get_initial_parameters(cfg.MODEL_NAME)
        )
        # The fl.server.strategy.FedProx in itself will not be different than FedAvg, the client needs to be adjusted.
        print("Using FedProx strategy.")
    else:
        strategy = fl.server.strategy.FedAvg(
            fraction_fit=1.0, fraction_evaluate=0.0,
            min_fit_clients=cfg.NUM_TRAIN_CLIENTS,
            min_available_clients=cfg.NUM_TRAIN_CLIENTS,
            evaluate_fn=evaluate_function,
            initial_parameters=get_initial_parameters(cfg.MODEL_NAME)

        )
        print(f"Using FedAvg strategy with {cfg.MODEL_NAME} model.")
    return strategy


#### External Attack

In [17]:

# ==============================================================================
#  DATA Distribution
# ==============================================================================

ied1b_train_files=[os.path.abspath(col) for col in modbus.dataset["benign_dataset_dir"] if col.find("ied1b")!=-1][:]
ied1a_train_files=[os.path.abspath(col) for col in modbus.dataset["benign_dataset_dir"] if col.find("ied1a")!=-1][:]
ied4c_train_files=[os.path.abspath(col) for col in modbus.dataset["benign_dataset_dir"] if col.find("ied4c")!=-1][:]
cent_agent_test_files=[os.path.abspath(col) for col in modbus.dataset["benign_dataset_dir"] if col.find("central-agent")!=-1][:]
# test_files=[os.path.abspath(col) for col in modbus.dataset["attack_dataset_dir"]["compromised-scada"] if col.find("ied1b")!=-1]
# ### missed attack logs files for the day 21 for ied1b which can reduce the accuracy.
# test_files.remove(dataset_directory+"attack/compromised-scada/ied1b/ied1b-network-captures/ready/vethc76bd3f-6-labeled.csv")
test_files=[os.path.abspath(col) for col in modbus.dataset["attack_dataset_dir"]["external"] if col.find("ied1a")!=-1][:1]
random.seed(42)
val_files = []

for list_files in [ied1b_train_files,ied1a_train_files,ied4c_train_files]: 
    random.shuffle(list_files)
    val_files += list_files[-2:]
TRAIN_CLIENT_DATA_MAPPING = {
    0: ied1b_train_files[:-2],
    1: ied1a_train_files[:-2],
    2: ied4c_train_files[:-2],
}

SERVER_EVALUATION_DATA_MAPPING = {
    0: val_files,
    1: test_files
}


In [18]:
len(ied1b_train_files),len(TRAIN_CLIENT_DATA_MAPPING[0]),len(val_files),len(test_files),len(val_files),test_files

(7,
 5,
 6,
 1,
 6,
 ['/home/hamid_rd3/labeling/FLBased-ICS-NIDS-main/ModbusDataset/attack/external/ied1a/ied1a-network-capture/ready/veth4edc015-0-labeled.csv'])

In [19]:

# ==============================================================================
#  CONFIGURATION: TWEAK  FEDERATED LEARNING EXPERIMENT
# ==============================================================================
class Config:
    """Global configuration class for the federated learning experiment."""
    # --- FL Parameters ---
    NUM_TRAIN_CLIENTS = 3
    NUM_ROUNDS = 15
    LOCAL_EPOCHS = 3
    BATCH_SIZE = 64
    LEARNING_RATE = 1e-4
    WEIGHT_DECAY = 1e-4
    # --- Strategy Selection ---
    # Choose from "FED_AVG", "FED_PROX"
    STRATEGY = "FED_AVG" 
    PROXIMAL_MU = 0.1 # Proximal term for FedProx
    # --- Model Selection ---
    # Choose from "AE" (Autoencoder) or "VAE" (Variational Autoencoder) or "AdverserialAutoencoder"
    MODEL_NAME = "AE"
    INPUT_DIM = 76
    # --- Anomaly Detection ---
    SHUFFLE_FILES=  True
# Instantiate the configuration
cfg = Config()

loaded_scalers = load_scalers("fitted_scalers")




Successfully loaded scalers for 'network-wide'


In [21]:
# ## cache all client data in memory
# client_fn=make_client_fn_with_cache()

# for i in range(3):
#     context= Context(0,i,{},0,0)
#     context.node_config["partition-id"]=i
#     client_fn(context)


In [ ]:

strategy=set_server_strategy()


history = fl.simulation.start_simulation(
    client_fn=client_function,
    num_clients=cfg.NUM_TRAIN_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=cfg.NUM_ROUNDS),
    strategy=strategy,
    client_resources={"num_cpus": 1, "num_gpus": 1} if DEVICE.type == "cuda" else {"num_cpus": 1},
)
print("Federated learning simulation finished.")

	Instead, use the `flwr run` CLI command to start a local simulation in your Flower app, as shown for example below:

		$ flwr new  # Create a new Flower app from a template

		$ flwr run  # Run the Flower app in Simulation Mode

	Using `start_simulation()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      Starting Flower simulation, config: num_rounds=15, no round_timeout


Using Autoencoder (AE) 
Using FedAvg strategy with AE model.


2025-07-23 22:59:43,873	INFO worker.py:1771 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'accelerator_type:G': 1.0, 'node:__internal_head__': 1.0, 'CPU': 4.0, 'memory': 5811432654.0, 'object_store_memory': 2905716326.0, 'GPU': 1.0, 'node:172.24.78.91': 1.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      Flower VCE: Resources for each Virtual Client: {'num_cpus': 4, 'num_gpus': 1}
INFO :      Flower VCE: Creating VirtualClientEngineActorPool with 1 actors
INFO :      [INIT]
INFO :      Using initial global parameters provided by strategy
INFO :      Starting evaluation of initial global parameters


Using Autoencoder (AE) 
--- Running Evaluation for Server round 0 ---
-----------mse_loss mean :  17.3566 std: 1.1771
Val: Accuracy: 0.1758  
 K: 1 K-SIGMA Threshold : ---thr 18.53
Counts of : original binary labels Counter({1: 65594, 0: 59838}) predicted binary labels Counter({0: 115438, 1: 9994})
Counts of  original  labels: {0: 59838, 1: 65556, 2: 1, 3: 1, 4: 1, 5: 2, 6: 32, 7: 1}
Counts of misclassified original labels: {0: 257, 1: 55836, 2: 1, 5: 1, 6: 18, 7: 1}
k= 1 ,Test : Accuracy: 0.5526 Recall : 0.1484 FDR: 0.0257  F1-score: 0.2576 
 K: 3 K-SIGMA Threshold : ---thr 20.89
Counts of : original binary labels Counter({1: 65594, 0: 59838}) predicted binary labels Counter({0: 122536, 1: 2896})
Counts of  original  labels: {0: 59838, 1: 65556, 2: 1, 3: 1, 4: 1, 5: 2, 6: 32, 7: 1}
Counts of misclassified original labels: {1: 62662, 2: 1, 3: 1, 4: 1, 5: 2, 6: 30, 7: 1}
k= 3 ,Test : Accuracy: 0.5001 Recall : 0.0442 FDR: 0.0000  F1-score: 0.0846 


INFO :      initial parameters (loss, other metrics): 17.799377351871932, {1: 'k= 1 ,Test : Accuracy: 0.5526 Recall : 0.1484 FDR: 0.0257  F1-score: 0.2576 ', 3: 'k= 3 ,Test : Accuracy: 0.5001 Recall : 0.0442 FDR: 0.0000  F1-score: 0.0846 '}
INFO :      
INFO :      [ROUND 1]
INFO :      configure_fit: strategy sampled 3 clients (out of 3)


...

(ClientAppActor pid=59036) Reusing cached dataloader for client 2...
(ClientAppActor pid=59036) Using Autoencoder (AE) 
(ClientAppActor pid=59036) Train : time 66.90 s Epoch 1
(ClientAppActor pid=59036) Train Loss: 0.0177
(ClientAppActor pid=59036) Train : time 61.23 s Epoch 2
(ClientAppActor pid=59036) Train Loss: 0.0146
(ClientAppActor pid=59036) Train : time 45.91 s Epoch 3
(ClientAppActor pid=59036) Train Loss: 0.0145
(ClientAppActor pid=59036) Train : time 52.45 s Epoch 4
(ClientAppActor pid=59036) Train Loss: 0.0145
(ClientAppActor pid=59036) Train : time 52.94 s Epoch 5
(ClientAppActor pid=59036) Train Loss: 0.0145
(ClientAppActor pid=59036) Train : time 52.69 s Epoch 6
(ClientAppActor pid=59036) Train Loss: 0.0145
(ClientAppActor pid=59036) Reusing cached dataloader for client 0...
(ClientAppActor pid=59036) Using Autoencoder (AE) 
(ClientAppActor pid=59036) Train : time 46.40 s Epoch 1
(ClientAppActor pid=59036) Train Loss: 0.0004
(ClientAppActor pid=59036) Train : time 22.60 

INFO :      aggregate_fit: received 3 results and 0 failures


Using Autoencoder (AE) 
(ClientAppActor pid=59036) Train : time 46.52 s Epoch 6
(ClientAppActor pid=59036) Train Loss: 0.0152
--- Running Evaluation for Server round 15 ---
-----------mse_loss mean :  2.0682 std: 4.3817
Val: Accuracy: 0.8870  
 K: 1 K-SIGMA Threshold : ---thr 6.45
Counts of : original binary labels Counter({1: 65594, 0: 59838}) predicted binary labels Counter({0: 93340, 1: 32092})
Counts of  original  labels: {0: 59838, 1: 65556, 2: 1, 3: 1, 4: 1, 5: 2, 6: 32, 7: 1}
Counts of misclassified original labels: {0: 6053, 1: 39546, 5: 1, 6: 8}
k= 1 ,Test : Accuracy: 0.6364 Recall : 0.3970 FDR: 0.1886  F1-score: 0.5331 
 K: 3 K-SIGMA Threshold : ---thr 15.21
Counts of : original binary labels Counter({1: 65594, 0: 59838}) predicted binary labels Counter({0: 121638, 1: 3794})
Counts of  original  labels: {0: 59838, 1: 65556, 2: 1, 3: 1, 4: 1, 5: 2, 6: 32, 7: 1}
Counts of misclassified original labels: {0: 3677, 1: 65439, 2: 1, 3: 1, 4: 1, 5: 2, 6: 32, 7: 1}
k= 3 ,Test : Accura

INFO :      fit progress: (15, 2.909872978984629, {1: 'k= 1 ,Test : Accuracy: 0.6364 Recall : 0.3970 FDR: 0.1886  F1-score: 0.5331 ', 3: 'k= 3 ,Test : Accuracy: 0.4487 Recall : 0.0018 FDR: 0.9692  F1-score: 0.0034 '}, 14808.52930966)
INFO :      configure_evaluate: no clients selected, skipping evaluation
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 15 round(s) in 14808.53s
INFO :      	History (loss, centralized):
INFO :      		round 0: 17.799377351871932
INFO :      		round 1: 4.68924048488424
INFO :      		round 2: 2.783328566474265
INFO :      		round 3: 2.5308967209324575
INFO :      		round 4: 2.918183009519102
INFO :      		round 5: 3.0478835145736336
INFO :      		round 6: 2.8576688962944066
INFO :      		round 7: 2.8403151209420243
INFO :      		round 8: 2.860112451368072
INFO :      		round 9: 2.7873132952516104
INFO :      		round 10: 2.8486131928056637
INFO :      		round 11: 2.9811623529083486
INFO :      		round 12: 2.8546660242202946
INFO :      		round 13

Federated learning simulation finished.


In [ ]:

strategy=set_server_strategy()

cfg.LEARNING_RATE=1e-4
history = fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=cfg.NUM_TRAIN_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=cfg.NUM_ROUNDS),
    strategy=strategy,
    client_resources={"num_cpus": 4, "num_gpus": 1} if DEVICE.type == "cuda" else {"num_cpus": 1},
)
print("Federated learning simulation finished.")

In [31]:

# Instantiate the configuration
cfg.STRATEGY="FED_PROX"
strategy=set_server_strategy()

# --- Start the Simulation ---
print("Starting federated learning simulation...")
history = fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=cfg.NUM_TRAIN_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=cfg.NUM_ROUNDS),
    strategy=strategy,
    client_resources={"num_cpus": 4, "num_gpus": 1} if DEVICE.type == "cuda" else {"num_cpus": 4},
)
print("Federated learning simulation finished.")

	Instead, use the `flwr run` CLI command to start a local simulation in your Flower app, as shown for example below:

		$ flwr new  # Create a new Flower app from a template

		$ flwr run  # Run the Flower app in Simulation Mode

	Using `start_simulation()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      Starting Flower simulation, config: num_rounds=15, no round_timeout


Using Autoencoder (AE) 
Using FedProx strategy.
Starting federated learning simulation...


2025-07-24 02:44:51,332	INFO worker.py:1771 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'accelerator_type:G': 1.0, 'node:__internal_head__': 1.0, 'CPU': 4.0, 'memory': 5876244480.0, 'object_store_memory': 2938122240.0, 'GPU': 1.0, 'node:172.24.78.91': 1.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      Flower VCE: Resources for each Virtual Client: {'num_cpus': 4, 'num_gpus': 1}
INFO :      Flower VCE: Creating VirtualClientEngineActorPool with 1 actors
INFO :      [INIT]
INFO :      Using initial global parameters provided by strategy
INFO :      Starting evaluation of initial global parameters


Using Autoencoder (AE) 
--- Running Evaluation for Server round 0 ---
-----------mse_loss mean :  16.8163 std: 1.0597
Val: Accuracy: 0.1758  
 K: 1 K-SIGMA Threshold : ---thr 17.88
Counts of : original binary labels Counter({1: 65594, 0: 59838}) predicted binary labels Counter({0: 112707, 1: 12725})
Counts of  original  labels: {0: 59838, 1: 65556, 2: 1, 3: 1, 4: 1, 5: 2, 6: 32, 7: 1}
Counts of misclassified original labels: {0: 6, 1: 52853, 2: 1, 5: 1, 6: 19, 7: 1}
k= 1 ,Test : Accuracy: 0.5784 Recall : 0.1939 FDR: 0.0005  F1-score: 0.3248 
 K: 3 K-SIGMA Threshold : ---thr 20.0
Counts of : original binary labels Counter({1: 65594, 0: 59838}) predicted binary labels Counter({0: 122536, 1: 2896})
Counts of  original  labels: {0: 59838, 1: 65556, 2: 1, 3: 1, 4: 1, 5: 2, 6: 32, 7: 1}
Counts of misclassified original labels: {1: 62662, 2: 1, 3: 1, 4: 1, 5: 2, 6: 30, 7: 1}
k= 3 ,Test : Accuracy: 0.5001 Recall : 0.0442 FDR: 0.0000  F1-score: 0.0846 


INFO :      initial parameters (loss, other metrics): 17.22125534153964, {1: 'k= 1 ,Test : Accuracy: 0.5784 Recall : 0.1939 FDR: 0.0005  F1-score: 0.3248 ', 3: 'k= 3 ,Test : Accuracy: 0.5001 Recall : 0.0442 FDR: 0.0000  F1-score: 0.0846 '}
INFO :      
INFO :      [ROUND 1]
INFO :      configure_fit: strategy sampled 3 clients (out of 3)


...

(ClientAppActor pid=109882) Reusing cached dataloader for client 0...
(ClientAppActor pid=109882) Using Autoencoder (AE) 
(ClientAppActor pid=109882) Train : time 70.54 s Epoch 1
(ClientAppActor pid=109882) Train Loss: 0.2067
(ClientAppActor pid=109882) Train : time 62.41 s Epoch 2
(ClientAppActor pid=109882) Train Loss: 0.2056
(ClientAppActor pid=109882) Train : time 62.86 s Epoch 3
(ClientAppActor pid=109882) Train Loss: 0.2056
(ClientAppActor pid=109882) Train : time 63.55 s Epoch 4
(ClientAppActor pid=109882) Train Loss: 0.2056
(ClientAppActor pid=109882) Train : time 62.62 s Epoch 5
(ClientAppActor pid=109882) Train Loss: 0.2056
(ClientAppActor pid=109882) Train : time 63.30 s Epoch 6
(ClientAppActor pid=109882) Train Loss: 0.2056
(ClientAppActor pid=109882) Reusing cached dataloader for client 1...
(ClientAppActor pid=109882) Using Autoencoder (AE) 
(ClientAppActor pid=109882) Train : time 107.09 s Epoch 1
(ClientAppActor pid=109882) Train Loss: 2.6268
(ClientAppActor pid=109882)

INFO :      aggregate_fit: received 3 results and 0 failures


Using Autoencoder (AE) 
(ClientAppActor pid=109882) Train : time 109.51 s Epoch 6
(ClientAppActor pid=109882) Train Loss: 2.6134
--- Running Evaluation for Server round 15 ---
-----------mse_loss mean :  2.3808 std: 1.8600
Val: Accuracy: 0.5792  
 K: 1 K-SIGMA Threshold : ---thr 4.241
Counts of : original binary labels Counter({1: 65594, 0: 59838}) predicted binary labels Counter({0: 117554, 1: 7878})
Counts of  original  labels: {0: 59838, 1: 65556, 2: 1, 3: 1, 4: 1, 5: 2, 6: 32, 7: 1}
Counts of misclassified original labels: {0: 4982, 1: 62662, 2: 1, 3: 1, 4: 1, 5: 2, 6: 30, 7: 1}
k= 1 ,Test : Accuracy: 0.4604 Recall : 0.0442 FDR: 0.6324  F1-score: 0.0788 
 K: 3 K-SIGMA Threshold : ---thr 7.961
Counts of : original binary labels Counter({1: 65594, 0: 59838}) predicted binary labels Counter({0: 123477, 1: 1955})
Counts of  original  labels: {0: 59838, 1: 65556, 2: 1, 3: 1, 4: 1, 5: 2, 6: 32, 7: 1}
Counts of misclassified original labels: {0: 1685, 1: 65286, 2: 1, 3: 1, 4: 1, 5: 2, 6: 

INFO :      fit progress: (15, 2.225313566314816, {1: 'k= 1 ,Test : Accuracy: 0.4604 Recall : 0.0442 FDR: 0.6324  F1-score: 0.0788 ', 3: 'k= 3 ,Test : Accuracy: 0.4658 Recall : 0.0041 FDR: 0.8619  F1-score: 0.0080 '}, 27416.896519448997)
INFO :      configure_evaluate: no clients selected, skipping evaluation
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 15 round(s) in 27416.90s
INFO :      	History (loss, centralized):
INFO :      		round 0: 17.22125534153964
INFO :      		round 1: 4.936402791951017
INFO :      		round 2: 2.5402404290771097
INFO :      		round 3: 2.303162271190765
INFO :      		round 4: 2.2426710787518336
INFO :      		round 5: 2.228197599097519
INFO :      		round 6: 2.22106350452835
INFO :      		round 7: 2.2240506808469926
INFO :      		round 8: 2.211386697573187
INFO :      		round 9: 2.22111682026915
INFO :      		round 10: 2.21812715256075
INFO :      		round 11: 2.2214185275687224
INFO :      		round 12: 2.22474727342305
INFO :      		round 13: 2.

Federated learning simulation finished.


### VAE

In [32]:
# Instantiate the configuration
cfg.STRATEGY="FED_AVG"
cfg.MODEL_NAME="VAE"
cfg.LEARNING_RATE=1e-4
strategy=set_server_strategy()

# --- Start the Simulation ---
print("Starting federated learning simulation...")
history = fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=cfg.NUM_TRAIN_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=cfg.NUM_ROUNDS),
    strategy=strategy,
    client_resources={"num_cpus": 4, "num_gpus": 1} if DEVICE.type == "cuda" else {"num_cpus": 4},
)
print("Federated learning simulation finished.")

	Instead, use the `flwr run` CLI command to start a local simulation in your Flower app, as shown for example below:

		$ flwr new  # Create a new Flower app from a template

		$ flwr run  # Run the Flower app in Simulation Mode

	Using `start_simulation()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      Starting Flower simulation, config: num_rounds=15, no round_timeout


Using Variational Autoencoder (VAE) 
Using FedAvg strategy with VAE model.
Starting federated learning simulation...


2025-07-24 09:40:57,914	INFO worker.py:1771 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'accelerator_type:G': 1.0, 'node:__internal_head__': 1.0, 'CPU': 4.0, 'memory': 5843900007.0, 'object_store_memory': 2921950003.0, 'GPU': 1.0, 'node:172.24.78.91': 1.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      Flower VCE: Resources for each Virtual Client: {'num_cpus': 4, 'num_gpus': 1}
INFO :      Flower VCE: Creating VirtualClientEngineActorPool with 1 actors
INFO :      [INIT]
INFO :      Using initial global parameters provided by strategy
INFO :      Starting evaluation of initial global parameters


Using Variational Autoencoder (VAE) 
--- Running Evaluation for Server round 0 ---
-----------mse_loss mean :  17.3588 std: 1.1775
Val: Accuracy: 0.1753  
 K: 1 K-SIGMA Threshold : ---thr 18.54
Counts of : original binary labels Counter({1: 65594, 0: 59838}) predicted binary labels Counter({0: 113634, 1: 11798})
Counts of  original  labels: {0: 59838, 1: 65556, 2: 1, 3: 1, 4: 1, 5: 2, 6: 32, 7: 1}
Counts of misclassified original labels: {0: 831, 1: 54600, 2: 1, 4: 1, 5: 1, 6: 24}
k= 1 ,Test : Accuracy: 0.5579 Recall : 0.1672 FDR: 0.0704  F1-score: 0.2834 
 K: 3 K-SIGMA Threshold : ---thr 20.89
Counts of : original binary labels Counter({1: 65594, 0: 59838}) predicted binary labels Counter({0: 122536, 1: 2896})
Counts of  original  labels: {0: 59838, 1: 65556, 2: 1, 3: 1, 4: 1, 5: 2, 6: 32, 7: 1}
Counts of misclassified original labels: {1: 62662, 2: 1, 3: 1, 4: 1, 5: 2, 6: 30, 7: 1}
k= 3 ,Test : Accuracy: 0.5001 Recall : 0.0442 FDR: 0.0000  F1-score: 0.0846 


INFO :      initial parameters (loss, other metrics): 17.795925282224633, {1: 'k= 1 ,Test : Accuracy: 0.5579 Recall : 0.1672 FDR: 0.0704  F1-score: 0.2834 ', 3: 'k= 3 ,Test : Accuracy: 0.5001 Recall : 0.0442 FDR: 0.0000  F1-score: 0.0846 '}
INFO :      
INFO :      [ROUND 1]
INFO :      configure_fit: strategy sampled 3 clients (out of 3)


model VAE is saved in fed_best_models/FED_AVG_VAE_f1_0.28_recall_0.17_.pth


(ClientAppActor pid=202739) /home/hamid_rd3/labeling/FLBased-ICS-NIDS-main/modbus.py:4: DeprecationWarning: 
(ClientAppActor pid=202739) Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(ClientAppActor pid=202739) (to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
(ClientAppActor pid=202739) but was not found to be installed on your system.
(ClientAppActor pid=202739) If this would cause problems for you,
(ClientAppActor pid=202739) please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
(ClientAppActor pid=202739)         
(ClientAppActor pid=202739)   import pandas as pd


(ClientAppActor pid=202739) Reusing cached dataloader for client 2...
(ClientAppActor pid=202739) Using Variational Autoencoder (VAE) 
(ClientAppActor pid=202739) Train : time 71.48 s Epoch 1
(ClientAppActor pid=202739) Train Loss: 1.2218
(ClientAppActor pid=202739) Train : time 57.39 s Epoch 2
(ClientAppActor pid=202739) Train Loss: 0.2097
(ClientAppActor pid=202739) Train : time 57.64 s Epoch 3
(ClientAppActor pid=202739) Train Loss: 0.1925
(ClientAppActor pid=202739) Train : time 57.69 s Epoch 4
(ClientAppActor pid=202739) Train Loss: 0.1818
(ClientAppActor pid=202739) Train : time 66.23 s Epoch 5
(ClientAppActor pid=202739) Train Loss: 0.1723
(ClientAppActor pid=202739) Train : time 61.25 s Epoch 6
(ClientAppActor pid=202739) Train Loss: 0.1676
(ClientAppActor pid=202739) Reusing cached dataloader for client 0...
(ClientAppActor pid=202739) Using Variational Autoencoder (VAE) 
(ClientAppActor pid=202739) Train : time 45.92 s Epoch 1
(ClientAppActor pid=202739) Train Loss: 1.2732
(C

KeyboardInterrupt: 

In [ ]:
cfg.STRATEGY="FED_PROX"
strategy=set_server_strategy()

print("Starting federated learning simulation...")
history = fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=cfg.NUM_TRAIN_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=cfg.NUM_ROUNDS),
    strategy=strategy,
    client_resources={"num_cpus": 4, "num_gpus": 1} if DEVICE.type == "cuda" else {"num_cpus": 4},
)
print("Federated learning simulation finished.")

	Instead, use the `flwr run` CLI command to start a local simulation in your Flower app, as shown for example below:

		$ flwr new  # Create a new Flower app from a template

		$ flwr run  # Run the Flower app in Simulation Mode

	Using `start_simulation()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      Starting Flower simulation, config: num_rounds=10, no round_timeout


Using Variational Autoencoder (VAE) 
Using FedProx strategy.
Starting federated learning simulation...


2025-07-23 05:28:07,393	INFO worker.py:1771 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'accelerator_type:G': 1.0, 'node:__internal_head__': 1.0, 'CPU': 4.0, 'memory': 5897306112.0, 'object_store_memory': 2948653056.0, 'GPU': 1.0, 'node:172.24.78.91': 1.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      Flower VCE: Resources for each Virtual Client: {'num_cpus': 4, 'num_gpus': 1}
INFO :      Flower VCE: Creating VirtualClientEngineActorPool with 1 actors
INFO :      [INIT]
INFO :      Using initial global parameters provided by strategy
INFO :      Starting evaluation of initial global parameters


Using Variational Autoencoder (VAE) 
--- Running Evaluation for Server round 0 ---
-----------mse_loss mean :  17.0129 std: 1.0944
Val: Accuracy: 0.1760  
 K: 1 K-SIGMA Threshold : ---thr 18.11
Counts of : original binary labels Counter({1: 65594, 0: 59838}) predicted binary labels Counter({0: 112655, 1: 12777})
Counts of  original  labels: {0: 59838, 1: 65556, 2: 1, 3: 1, 4: 1, 5: 2, 6: 32, 7: 1}
Counts of misclassified original labels: {0: 305, 1: 53103, 2: 1, 5: 1, 6: 16, 7: 1}
k= 1 ,Test : Accuracy: 0.5741 Recall : 0.1901 FDR: 0.0239  F1-score: 0.3183 
 K: 3 K-SIGMA Threshold : ---thr 20.3
Counts of : original binary labels Counter({1: 65594, 0: 59838}) predicted binary labels Counter({0: 122536, 1: 2896})
Counts of  original  labels: {0: 59838, 1: 65556, 2: 1, 3: 1, 4: 1, 5: 2, 6: 32, 7: 1}
Counts of misclassified original labels: {1: 62662, 2: 1, 3: 1, 4: 1, 5: 2, 6: 30, 7: 1}
k= 3 ,Test : Accuracy: 0.5001 Recall : 0.0442 FDR: 0.0000  F1-score: 0.0846 


INFO :      initial parameters (loss, other metrics): 17.437362475285415, {1: 'k= 1 ,Test : Accuracy: 0.5741 Recall : 0.1901 FDR: 0.0239  F1-score: 0.3183 ', 3: 'k= 3 ,Test : Accuracy: 0.5001 Recall : 0.0442 FDR: 0.0000  F1-score: 0.0846 '}
INFO :      
INFO :      [ROUND 1]
INFO :      configure_fit: strategy sampled 3 clients (out of 3)


...

(ClientAppActor pid=93706) Using Variational Autoencoder (VAE) 


ERROR :     Traceback (most recent call last):
  File "/home/hamid_rd3/labeling/Project1404/FLBased-ICS-NIDS/vnv/lib/python3.10/site-packages/flwr/simulation/ray_transport/ray_client_proxy.py", line 95, in _submit_job
    out_mssg, updated_context = self.actor_pool.get_client_result(
  File "/home/hamid_rd3/labeling/Project1404/FLBased-ICS-NIDS/vnv/lib/python3.10/site-packages/flwr/simulation/ray_transport/ray_actor.py", line 401, in get_client_result
    return self._fetch_future_result(cid)
  File "/home/hamid_rd3/labeling/Project1404/FLBased-ICS-NIDS/vnv/lib/python3.10/site-packages/flwr/simulation/ray_transport/ray_actor.py", line 282, in _fetch_future_result
    res_cid, out_mssg, updated_context = ray.get(
  File "/home/hamid_rd3/labeling/Project1404/FLBased-ICS-NIDS/vnv/lib/python3.10/site-packages/ray/_private/auto_init_hook.py", line 21, in auto_init_wrapper
    return fn(*args, **kwargs)
  File "/home/hamid_rd3/labeling/Project1404/FLBased-ICS-NIDS/vnv/lib/python3.10/site-pac

Using Variational Autoencoder (VAE) 
--- Running Evaluation for Server round 10 ---
-----------mse_loss mean :  17.0132 std: 1.0941
Val: Accuracy: 0.1760  
 K: 1 K-SIGMA Threshold : ---thr 18.11
Counts of : original binary labels Counter({1: 65594, 0: 59838}) predicted binary labels Counter({0: 112736, 1: 12696})
Counts of  original  labels: {0: 59838, 1: 65556, 2: 1, 3: 1, 4: 1, 5: 2, 6: 32, 7: 1}
Counts of misclassified original labels: {0: 306, 1: 53186, 2: 1, 5: 1, 6: 15, 7: 1}
k= 1 ,Test : Accuracy: 0.5734 Recall : 0.1889 FDR: 0.0241  F1-score: 0.3165 
 K: 3 K-SIGMA Threshold : ---thr 20.3
Counts of : original binary labels Counter({1: 65594, 0: 59838}) predicted binary labels Counter({0: 122536, 1: 2896})
Counts of  original  labels: {0: 59838, 1: 65556, 2: 1, 3: 1, 4: 1, 5: 2, 6: 32, 7: 1}
Counts of misclassified original labels: {1: 62662, 2: 1, 3: 1, 4: 1, 5: 2, 6: 30, 7: 1}
k= 3 ,Test : Accuracy: 0.5001 Recall : 0.0442 FDR: 0.0000  F1-score: 0.0846 


INFO :      fit progress: (10, 17.437284743924995, {1: 'k= 1 ,Test : Accuracy: 0.5734 Recall : 0.1889 FDR: 0.0241  F1-score: 0.3165 ', 3: 'k= 3 ,Test : Accuracy: 0.5001 Recall : 0.0442 FDR: 0.0000  F1-score: 0.0846 '}, 324.71736956899986)
INFO :      configure_evaluate: no clients selected, skipping evaluation
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 10 round(s) in 324.72s
INFO :      	History (loss, centralized):
INFO :      		round 0: 17.437362475285415
INFO :      		round 1: 17.43757573824861
INFO :      		round 2: 17.437021653166656
INFO :      		round 3: 17.43768535939792
INFO :      		round 4: 17.437864739460426
INFO :      		round 5: 17.43802020218126
INFO :      		round 6: 17.437769070093754
INFO :      		round 7: 17.437759104534727
INFO :      		round 8: 17.437480068881943
INFO :      		round 9: 17.437970374386122
INFO :      		round 10: 17.437284743924995
INFO :      	History (metrics, centralized):
INFO :      	{1: [(0,
INFO :      	      'k= 1 ,Test : Acc

Federated learning simulation finished.


In [ ]:
# Instantiate the configuration
cfg.STRATEGY="FED_AVG"
cfg.MODEL_NAME="AAE"
cfg.LEARNING_RATE=1e-4
strategy=set_server_strategy()

# --- Start the Simulation ---
print("Starting federated learning simulation...")
history = fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=cfg.NUM_TRAIN_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=cfg.NUM_ROUNDS),
    strategy=strategy,
    client_resources={"num_cpus": 4, "num_gpus": 1} if DEVICE.type == "cuda" else {"num_cpus": 4},
)
print("Federated learning simulation finished.")

	Instead, use the `flwr run` CLI command to start a local simulation in your Flower app, as shown for example below:

		$ flwr new  # Create a new Flower app from a template

		$ flwr run  # Run the Flower app in Simulation Mode

	Using `start_simulation()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      Starting Flower simulation, config: num_rounds=10, no round_timeout


Using Adverserial Autoencoder (AAE) 
Using FedAvg strategy with AAE model.
Starting federated learning simulation...


2025-07-23 05:33:35,570	INFO worker.py:1771 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'accelerator_type:G': 1.0, 'node:__internal_head__': 1.0, 'CPU': 4.0, 'object_store_memory': 2929537843.0, 'memory': 5859075687.0, 'GPU': 1.0, 'node:172.24.78.91': 1.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      Flower VCE: Resources for each Virtual Client: {'num_cpus': 4, 'num_gpus': 1}
INFO :      Flower VCE: Creating VirtualClientEngineActorPool with 1 actors
INFO :      [INIT]
INFO :      Using initial global parameters provided by strategy
INFO :      Starting evaluation of initial global parameters


Using Adverserial Autoencoder (AAE) 
--- Running Evaluation for Server round 0 ---
-----------mse_loss mean :  17.2600 std: 1.1154
Val: Accuracy: 0.1758  
 K: 1 K-SIGMA Threshold : ---thr 18.38
Counts of : original binary labels Counter({1: 65594, 0: 59838}) predicted binary labels Counter({0: 113419, 1: 12013})
Counts of  original  labels: {0: 59838, 1: 65556, 2: 1, 3: 1, 4: 1, 5: 2, 6: 32, 7: 1}
Counts of misclassified original labels: {0: 262, 1: 53819, 2: 1, 5: 1, 6: 21, 7: 1}
k= 1 ,Test : Accuracy: 0.5687 Recall : 0.1791 FDR: 0.0218  F1-score: 0.3028 
 K: 3 K-SIGMA Threshold : ---thr 20.61
Counts of : original binary labels Counter({1: 65594, 0: 59838}) predicted binary labels Counter({0: 122536, 1: 2896})
Counts of  original  labels: {0: 59838, 1: 65556, 2: 1, 3: 1, 4: 1, 5: 2, 6: 32, 7: 1}
Counts of misclassified original labels: {1: 62662, 2: 1, 3: 1, 4: 1, 5: 2, 6: 30, 7: 1}
k= 3 ,Test : Accuracy: 0.5001 Recall : 0.0442 FDR: 0.0000  F1-score: 0.0846 


INFO :      initial parameters (loss, other metrics): 17.676394381019197, {1: 'k= 1 ,Test : Accuracy: 0.5687 Recall : 0.1791 FDR: 0.0218  F1-score: 0.3028 ', 3: 'k= 3 ,Test : Accuracy: 0.5001 Recall : 0.0442 FDR: 0.0000  F1-score: 0.0846 '}
INFO :      
INFO :      [ROUND 1]
INFO :      configure_fit: strategy sampled 3 clients (out of 3)


...

(ClientAppActor pid=95436) Using Adverserial Autoencoder (AAE) 
(ClientAppActor pid=95436) Train : time 64.14 s Epoch 1
(ClientAppActor pid=95436) Generator Loss: 0.2329 Discriminator Loss: 49.6335
(ClientAppActor pid=95436) Train : time 48.91 s Epoch 2
(ClientAppActor pid=95436) Generator Loss: 0.1687 Discriminator Loss: 30.7160
(ClientAppActor pid=95436) Train : time 65.54 s Epoch 3
(ClientAppActor pid=95436) Generator Loss: 0.1241 Discriminator Loss: 47.0797
(ClientAppActor pid=95436) Train : time 55.92 s Epoch 4
(ClientAppActor pid=95436) Generator Loss: 0.0923 Discriminator Loss: 37.2568
(ClientAppActor pid=95436) Train : time 48.30 s Epoch 5
(ClientAppActor pid=95436) Generator Loss: 0.1221 Discriminator Loss: 31.0596
(ClientAppActor pid=95436) Train : time 57.99 s Epoch 6
(ClientAppActor pid=95436) Generator Loss: 0.1039 Discriminator Loss: 35.2718
(ClientAppActor pid=95436) Using Adverserial Autoencoder (AAE) 
(ClientAppActor pid=95436) Train : time 97.32 s Epoch 1
(ClientAppAc

INFO :      aggregate_fit: received 3 results and 0 failures


(ClientAppActor pid=95436) Train : time 87.50 s Epoch 6
(ClientAppActor pid=95436) Generator Loss: 0.4570 Discriminator Loss: 22.7955
Using Adverserial Autoencoder (AAE) 
--- Running Evaluation for Server round 10 ---
-----------mse_loss mean :  1.3985 std: 1.8151
Val: Accuracy: 0.6796  
 K: 1 K-SIGMA Threshold : ---thr 3.214
Counts of : original binary labels Counter({1: 65594, 0: 59838}) predicted binary labels Counter({0: 76691, 1: 48741})
Counts of  original  labels: {0: 59838, 1: 65556, 2: 1, 3: 1, 4: 1, 5: 2, 6: 32, 7: 1}
Counts of misclassified original labels: {0: 23355, 1: 40186, 3: 1, 4: 1, 5: 2, 6: 17, 7: 1}
k= 1 ,Test : Accuracy: 0.4932 Recall : 0.3870 FDR: 0.4792  F1-score: 0.4441 
 K: 3 K-SIGMA Threshold : ---thr 6.844
Counts of : original binary labels Counter({1: 65594, 0: 59838}) predicted binary labels Counter({0: 122536, 1: 2896})
Counts of  original  labels: {0: 59838, 1: 65556, 2: 1, 3: 1, 4: 1, 5: 2, 6: 32, 7: 1}
Counts of misclassified original labels: {1: 62662,

INFO :      fit progress: (10, 1.8803939833933925, {1: 'k= 1 ,Test : Accuracy: 0.4932 Recall : 0.3870 FDR: 0.4792  F1-score: 0.4441 ', 3: 'k= 3 ,Test : Accuracy: 0.5001 Recall : 0.0442 FDR: 0.0000  F1-score: 0.0846 '}, 16095.926983861998)
INFO :      configure_evaluate: no clients selected, skipping evaluation
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 10 round(s) in 16095.93s
INFO :      	History (loss, centralized):
INFO :      		round 0: 17.676394381019197
INFO :      		round 1: 2.5080688640059954
INFO :      		round 2: 1.0996081293051214
INFO :      		round 3: 1.6815560423177498
INFO :      		round 4: 2.9950583284169907
INFO :      		round 5: 1.3847850578401046
INFO :      		round 6: 2.1445424612539066
INFO :      		round 7: 1.3201321233815932
INFO :      		round 8: 1.4599423143615664
INFO :      		round 9: 2.250904125342815
INFO :      		round 10: 1.8803939833933925
INFO :      	History (metrics, centralized):
INFO :      	{1: [(0,
INFO :      	      'k= 1 ,Test :

Federated learning simulation finished.


In [ ]:
# Instantiate the configuration
cfg.STRATEGY="FED_PROX"
cfg.MODEL_NAME="AAE"
cfg.LEARNING_RATE=1e-4
strategy=set_server_strategy()

# --- Start the Simulation ---
print("Starting federated learning simulation...")
history = fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=cfg.NUM_TRAIN_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=cfg.NUM_ROUNDS),
    strategy=strategy,
    client_resources={"num_cpus": 4, "num_gpus": 1/cfg.NUM_TRAIN_CLIENTS} if DEVICE.type == "cuda" else {"num_cpus": 4},
)
print("Federated learning simulation finished.")

	Instead, use the `flwr run` CLI command to start a local simulation in your Flower app, as shown for example below:

		$ flwr new  # Create a new Flower app from a template

		$ flwr run  # Run the Flower app in Simulation Mode

	Using `start_simulation()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      Starting Flower simulation, config: num_rounds=10, no round_timeout


Using Adverserial Autoencoder (AAE) 
Using FedProx strategy.
Starting federated learning simulation...


2025-07-23 09:37:59,852	INFO worker.py:1771 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'accelerator_type:G': 1.0, 'node:__internal_head__': 1.0, 'CPU': 4.0, 'object_store_memory': 2918409830.0, 'memory': 5836819662.0, 'GPU': 1.0, 'node:172.24.78.91': 1.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      Flower VCE: Resources for each Virtual Client: {'num_cpus': 4, 'num_gpus': 0.3333333333333333}
INFO :      Flower VCE: Creating VirtualClientEngineActorPool with 1 actors
INFO :      [INIT]
INFO :      Using initial global parameters provided by strategy
INFO :      Starting evaluation of initial global parameters


Using Adverserial Autoencoder (AAE) 
--- Running Evaluation for Server round 0 ---
-----------mse_loss mean :  16.6561 std: 1.2226
Val: Accuracy: 0.1758  
 K: 1 K-SIGMA Threshold : ---thr 17.88
Counts of : original binary labels Counter({1: 65594, 0: 59838}) predicted binary labels Counter({0: 118019, 1: 7413})
Counts of  original  labels: {0: 59838, 1: 65556, 2: 1, 3: 1, 4: 1, 5: 2, 6: 32, 7: 1}
Counts of misclassified original labels: {0: 259, 1: 58405, 2: 1, 3: 1, 4: 1, 5: 1, 6: 30, 7: 1}
k= 1 ,Test : Accuracy: 0.5320 Recall : 0.1091 FDR: 0.0349  F1-score: 0.1960 
 K: 3 K-SIGMA Threshold : ---thr 20.32
Counts of : original binary labels Counter({1: 65594, 0: 59838}) predicted binary labels Counter({0: 122536, 1: 2896})
Counts of  original  labels: {0: 59838, 1: 65556, 2: 1, 3: 1, 4: 1, 5: 2, 6: 32, 7: 1}
Counts of misclassified original labels: {1: 62662, 2: 1, 3: 1, 4: 1, 5: 2, 6: 30, 7: 1}
k= 3 ,Test : Accuracy: 0.5001 Recall : 0.0442 FDR: 0.0000  F1-score: 0.0846 
model Adversari

INFO :      initial parameters (loss, other metrics): 17.07131951973978, {1: 'k= 1 ,Test : Accuracy: 0.5320 Recall : 0.1091 FDR: 0.0349  F1-score: 0.1960 ', 3: 'k= 3 ,Test : Accuracy: 0.5001 Recall : 0.0442 FDR: 0.0000  F1-score: 0.0846 '}
INFO :      
INFO :      [ROUND 1]
INFO :      configure_fit: strategy sampled 3 clients (out of 3)
(ClientAppActor pid=151015) /home/hamid_rd3/labeling/FLBased-ICS-NIDS-main/modbus.py:4: DeprecationWarning: 
(ClientAppActor pid=151015) Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(ClientAppActor pid=151015) (to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
(ClientAppActor pid=151015) but was not found to be installed on your system.
(ClientAppActor pid=151015) If this would cause problems for you,
(ClientAppActor pid=151015) please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
(ClientAppActor pid=15

...

model AdversarialAutoencoder is saved in fed_models/AdversarialAutoencoder_f1_1.00_recall_1.00_.pth
(ClientAppActor pid=151015) Using Adverserial Autoencoder (AAE) 
(ClientAppActor pid=151015) Train : time 319.25 s Epoch 1
(ClientAppActor pid=151015) Generator Loss: 0.7438 Discriminator Loss: 13.1829
(ClientAppActor pid=151015) Train : time 328.17 s Epoch 2
(ClientAppActor pid=151015) Generator Loss: 0.7426 Discriminator Loss: 13.3813
(ClientAppActor pid=151015) Train : time 314.04 s Epoch 3
(ClientAppActor pid=151015) Generator Loss: 0.7427 Discriminator Loss: 14.0335
(ClientAppActor pid=151015) Train : time 233.78 s Epoch 4
(ClientAppActor pid=151015) Generator Loss: 0.7538 Discriminator Loss: 13.7597
(ClientAppActor pid=151015) Train : time 167.94 s Epoch 5
(ClientAppActor pid=151015) Generator Loss: 0.7712 Discriminator Loss: 14.3026
(ClientAppActor pid=151015) Train : time 175.46 s Epoch 6
(ClientAppActor pid=151015) Generator Loss: 0.7558 Discriminator Loss: 15.1063
(ClientAppAct

INFO :      aggregate_fit: received 3 results and 0 failures


Using Adverserial Autoencoder (AAE) 
--- Running Evaluation for Server round 5 ---
(ClientAppActor pid=151015) Train : time 96.63 s Epoch 6
(ClientAppActor pid=151015) Generator Loss: 0.4493 Discriminator Loss: 35.5159
-----------mse_loss mean :  0.0514 std: 0.1226
Val: Accuracy: 0.6758  
 K: 1 K-SIGMA Threshold : ---thr 0.174
Counts of : original binary labels Counter({1: 65594, 0: 59838}) predicted binary labels Counter({1: 65801, 0: 59631})
Counts of  original  labels: {0: 59838, 1: 65556, 2: 1, 3: 1, 4: 1, 5: 2, 6: 32, 7: 1}
Counts of misclassified original labels: {0: 372, 1: 165}
k= 1 ,Test : Accuracy: 0.9957 Recall : 0.9975 FDR: 0.0057  F1-score: 0.9959 
 K: 3 K-SIGMA Threshold : ---thr 0.4193
Counts of : original binary labels Counter({1: 65594, 0: 59838}) predicted binary labels Counter({0: 88515, 1: 36917})
Counts of  original  labels: {0: 59838, 1: 65556, 2: 1, 3: 1, 4: 1, 5: 2, 6: 32, 7: 1}
Counts of misclassified original labels: {0: 147, 1: 28824}
k= 3 ,Test : Accuracy: 0

INFO :      fit progress: (5, 0.8742645417437337, {1: 'k= 1 ,Test : Accuracy: 0.9957 Recall : 0.9975 FDR: 0.0057  F1-score: 0.9959 ', 3: 'k= 3 ,Test : Accuracy: 0.7690 Recall : 0.5606 FDR: 0.0040  F1-score: 0.7174 '}, 15592.369933458001)
INFO :      configure_evaluate: no clients selected, skipping evaluation
INFO :      
INFO :      [ROUND 6]
INFO :      configure_fit: strategy sampled 3 clients (out of 3)


(ClientAppActor pid=151015) Using Adverserial Autoencoder (AAE) 
(ClientAppActor pid=151015) Train : time 96.86 s Epoch 1
(ClientAppActor pid=151015) Generator Loss: 0.3471 Discriminator Loss: 25.5864
(ClientAppActor pid=151015) Train : time 98.45 s Epoch 2
(ClientAppActor pid=151015) Generator Loss: 0.2632 Discriminator Loss: 23.9524
(ClientAppActor pid=151015) Train : time 95.82 s Epoch 3
(ClientAppActor pid=151015) Generator Loss: 0.3389 Discriminator Loss: 19.5536
(ClientAppActor pid=151015) Train : time 95.44 s Epoch 4
(ClientAppActor pid=151015) Generator Loss: 0.2413 Discriminator Loss: 24.3305
(ClientAppActor pid=151015) Train : time 99.83 s Epoch 5
(ClientAppActor pid=151015) Generator Loss: 0.2546 Discriminator Loss: 24.5307
(ClientAppActor pid=151015) Train : time 95.86 s Epoch 6
(ClientAppActor pid=151015) Generator Loss: 0.3326 Discriminator Loss: 30.8307
(ClientAppActor pid=151015) Using Adverserial Autoencoder (AAE) 
(ClientAppActor pid=151015) Train : time 148.89 s Epoc